In [1]:
from koselleck import *

In [2]:
def classify_vector_change(row,perc_threshold=75):
    name=row.vector
    ns=str(name).split('.')[0].split('-')
    n1,n2=ns[0],ns[-1]
    res=f'{n1}~{n2}'
    if row.mean_diff_p<0.05 and row.perc>perc_threshold:
        res=f'+{n1}' if row.mean_diff>0 else (f'+{n2}' if n2!=n1 else f'-{n1}')
#         new=(row.mean2>1 and row.mean1<1) if row.mean_diff>0 else (row.mean2<-1 and row.mean1>-1)
        new=(row.mean2>1 and row.mean1<0.75) if row.mean_diff>0 else (row.mean2<-1 and row.mean1>-0.75)
        if new: res=res+'+'
    return res

In [3]:
df=get_dfchange_from_decade_data(wide=False).reset_index()
df['perc']=df.mean_diff_t_abs.apply(lambda x: percentileofscore(df.mean_diff_t_abs, x))
# df

dfchsz=get_dfchange_from_sattelzeit_models().reset_index()
remove_words = set(df.word) - set(dfchsz.word)
df=df[~df.word.isin(remove_words)]
# remove_words=set()
df['class_vec_change']=df.progress_apply(classify_vector_change,axis=1)
dfchsz['class_vec_change']=dfchsz['class_change']
df=df.append(dfchsz)
df.to_csv('data/data.word_changes_classed.csv')
df

100%|██████████| 80420/80420 [00:01<00:00, 43086.54it/s]


,vector,word,mean1,mean2,mean_diff,mean_diff_t,mean_diff_p,mean_diff_t_abs,len1,len2,...,score_diff_abstractness,is_clean_noiseaware,neighborhood1_local,neighborhood2_local,score_diff_t_abstractness,score_diff_p_abstractness,class_abs,class_change,class_signif,class
0,Sing-Plural.M,patronage,-0.204667,0.334828,0.539496,-4.834986e+16,0.000000e+00,4.834986e+16,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sing-Plural.M,guile,-0.334521,-0.340682,-0.006161,3.491972e+14,0.000000e+00,3.491972e+14,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sing-Plural.M,craftsman,0.187390,-0.369967,-0.557357,6.825422e+02,0.000000e+00,6.825422e+02,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sing-Plural.M,haft,-0.062734,-0.362382,-0.299648,5.731582e+02,6.788169e-316,5.731582e+02,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sing-Plural.M,task,-0.162905,0.928315,1.091221,-3.289679e+02,1.569778e-263,3.289679e+02,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4819,NaN,love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.002294,1.0,"lover (10), passion (10), pity (10), rapture (...","affection (10), passion (10), tenderness (10),...",0.159039,0.875409,Abs~Conc,-Changed,False,Abs~Conc -Changed
4820,NaN,grief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.016623,1.0,"sorrow (10), anguish (10), sigh (10), woe (10)...","sorrow (10), anguish (10), remorse (10), sadne...",-0.830267,0.417264,Abs~Conc,-Changed,False,Abs~Conc -Changed
4821,NaN,truth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.008670,1.0,"falsehood (10), sincerity (10), fallacy (10), ...","revelation (10), falsehood (10), sincerity (10...",-0.718520,0.481660,Abs~Conc,-Changed,False,Abs~Conc -Changed
4822,NaN,translation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.043636,1.0,"version (10), translator (10), fragment (10), ...","version (10), translator (10), text (10), lexi...",-2.287025,0.034522,Abs~Conc,-Changed,False,Abs~Conc -Changed


In [4]:
df[df.word=='culture'].class_vec_change

839                  +Pos
897               +Virtue
3239              +Interp
4092              +Strong
5621               +Human
7123                +Acad
10583               +Abs+
11084           +Pleasure
29175      Active~Passive
30457           Woman~Man
31388           Freq~Freq
40376         Ambig~Ambig
40504          Time~Space
70138          Qual~Quant
71903    Collective~Indiv
74317           Judg~Perc
440              +Changed
Name: class_vec_change, dtype: object

In [5]:
def format_words(wdf,joinby=', ',lim=None,randomize=False):
    wdf=wdf.groupby('word').mean().reset_index()
#     wdf=wdf.sort_values('word')
    wdf=wdf.sort_values('perc',ascending=False)
    words=wdf.drop_duplicates('word').word
    if randomize: words=words.sample(frac=1)
    o=joinby.join([str(x) for x in words.iloc[:lim]])
    if lim and len(words)>lim: o+=f' ...'
    if len(words): o+=f' ({len(words)})'
    return o

In [6]:

def get_vec_change_table(df,add_solo_col=True, remove_noisy=True, remove_words=set(),lim=None):
    if remove_noisy:
        dfchsz = get_dfchange_from_sattelzeit_models()
        remove_words|=set(dfchsz.query('is_clean_noiseaware!=True').index)
    
    ld=[]
    cs=[c for c in df.class_vec_change.unique() if '~' not in c]
    for c1 in tqdm(cs):
        c1w=df[df.class_vec_change==c1].word
        added=False
        for c2 in cs:
            if c2==c1: continue
            c2w=df[df.class_vec_change==c2].word        
            sw=(set(c1w) & set(c2w))
            sw-=remove_words
            cw=df[df.class_vec_change.isin({c1,c2}) & df.word.isin(sw)]
            ostr=format_words(cw,lim=lim)
            ld.append({'words':ostr,'change1':c1,'change2':c2,'size':len(sw)})
            added=True
        if add_solo_col and not added: ld.append({'words':', '.join(c1w),'change1':c1,'change2':'*','size':len(c1w)})
    odf=pd.DataFrame(ld)
    return odf.pivot('change1','change2','words').fillna('')
    # odf        

In [7]:
# words_ever_abs = set(df.query('vector=="Abs-Conc.Median" & (mean1>1 | mean2>1)').word)
# words_ever_abs

In [8]:
odf=get_vec_change_table(
    df,#[df.word.isin(words_ever_abs)],
    remove_noisy=False,
    lim=200,
#     remove_words=remove_words
)
# odf

100%|██████████| 69/69 [00:49<00:00,  1.38it/s]


In [11]:
odf.loc['+Sing']

change2
+Abs        burst, breadth, elder, gallery, chairman, hera...
+Abs+       tact, exaggeration, annoyance, organization, m...
+Acad       accommodation, possession, enjoyment, advance,...
+Acad+      value, process, experience, department, abilit...
+Active     shape, task, instance, acquisition, understand...
                                  ...                        
+Woman+     refinement, attraction, prayer, nursery, growt...
-Ambig      exaggeration, muslin, pantomime, handkerchief,...
-Ambig+     shade, supper, waistcoat, dialect, chalk, temp...
-Changed    reviewer, arrest, fob, distribution, leech, bl...
-Freq          inaccuracy, turnip, zephyr, felon, evasion (5)
Name: +Sing, Length: 69, dtype: object

In [9]:
printm(
    odf[
        ['+Abs+','+Abs','+Conc','+Conc+','+Changed','-Changed']
    ].loc[
        [c for c in odf.index]# if c.startswith('++')]# if c.endswith('+')]
    ].to_markdown()
)

| change1      | +Abs+                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | +Abs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | +Conc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | +Conc+                                                                                                                                                                                                                                                                                         | +Changed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | -Changed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|:-------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| +Abs         |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                | feud, childe, longing, lecturer, caring, clue, herald, serf, thinker, germ, chef, ache, seer, semi, role, rote, sig, negro, papacy, convulsion, goldsmith, dwelling, anticipation, discouragement, renunciation, cheque, forte, rite, commune, gurney, guild, derby, graduate, trash, closing, aid, enigma, thrill, budget, sob, lunatic, duo, wail, meantime, comp, debility, turkey, reprobation, bey, communication, sine, remonstrance, retort, mater, stoppage, sharpness, breadth, array, moralist, note, blockade, labor, urge, abstinence, harp, humiliation, peninsula, banking, collector, star, ballot, data, choosing, retribution, lodger, gateway, recitative, forum, haven, victor, dissent, labourer, embryo, transformation, ringing, partnership, engineer, gilt, basil, parentage, workhouse, lane, intervention, builder, ending, suggestion, obstruction, jubilee, volunteer, din, mourning, arbitration, demolition, marquess, reckoning, stipend, chronicle, vita, street, opus, scaffold, moor, monday, divorce, relaxation, saturday, vein, entry, tuesday, accounting, jail, wednesday, gazette, duel, promoter, thursday, quid (127)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | regeneration, retaliation, goo, blight, valentine, lip, indigestion, reservation, liar, liver, democracy, lamb, tinge, cross, infusion, adjournment, mediocrity, bishopric, tariff, fusion, sunday, freeman, rider, unwillingness, cycle, refutation, shoemaker, acquittal, gentlewoman, implication, thesis, prediction, valuation, elder, perversion, intuition, sound, liter, park, echo, exaltation, tenant, epilogue, lace, warren, banker, prologue, ado, deafness, celibacy, judaism, bearer, comrade, dit, exclusion, lint, baker, extinction, panic, evasion, narrowness, chairman, episode, fox, dollar, commissioner, remission, meddling, gall, allowance, bombardment, inquest, explosion, bloodshed, disaffection, occult, gaol, vestry, paradox, chaplain, rudeness, butler, punch, acquiescence, ebb, stability, prohibition, interpreting, distrust, biographer, coquette, inferiority, willingness, inexperience, burst, concession, recorder, disapprobation, condemnation, creed, denial, coroner, ire, flash, diocese, argyle, explanation, faithfulness, believer, universality, expansion, coming, revolt, focus, courtship, mademoiselle, advocate, impertinence, canto, chill, melancholy, comprehension, hardship, consistency, unhappiness, cathedral, assent, scepticism, apoplexy, gladness, misrepresentation, limitation, bid, fanaticism, kinsman, barrister, friday, region, niece, parody, dean, intoxication, nerve, tide, brandy, bologna, pang, sales, theory, blindness, viscount, rector, app, square, aurora, surveyor, antipathy, priesthood, vocabulary, blasphemy, painter, tea, relict, impeachment, widower, broker, bitterness, misconduct, vicar, insensibility, afternoon, epidemic, hierarchy, nightingale, inconsistency, walker, edification, gentile, incarnation, frankness, rabbi, brevity, ear, storm, eon, vault, intemperance, banishment, corps, executor, calmness, penny, week, countess, plaintiff, apothecary, infirmary, depth, reliance, deputy ... (287)                                                    |
| +Abs+        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                | regime, drawback, loathing, canning, annoyance, retention, veto, congratulation, trait, goodwill, mania, hostility, supporter, lunacy, viewing, endowment, advert, cessation, consummation, delineation, advent, incorporation, confiscation, threat, desertion, ruler, privation, mood, boon, presentment, lending, greed, licensing, deduction, irritability, craving, auditor, sect, imbecility, disaster, semblance, symptom, lifetime, culture, detention, variance, accord, seizure, compulsion, absence, import, agitation, suppression, scrutiny, sneer, par, formula, grace, irony, exhaust, taint, manifesto, mover, piracy, destiny, delegate, ego, patient, peril, chagrin, intrigue, correction, emphasis, hearing, solemnity, currency, longevity, heroine, abbe, warning, strife (81)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | deception, appropriation, layman, exaggeration, martyrdom, stagnation, hearer, arrest, checking, distraction, rebuke, enthusiast, inaccuracy, reformer, axiom, irritation, follower, churchman, sadness, carelessness, accomplice, sphere, learner, default, fervour, bane, recoil, proclamation, prosecutor, sacrilege, delirium, penalty, verdict, courtesy, consecration, coldness, value, menace, art, critique, hindrance, mortification, relapse, interpreter, contentment, magnitude, abatement, formality, license, impetuosity, indictment, archdeacon, absolution, phraseology, fluency, visitation, impediment, bankruptcy, massacre, ordination, champion, lord, comment, weariness, command, disciple, stress, disorder, canon, reluctance, confessor, exposition, malady, interruption, emotion, outcry, digression, philology, defiance, trial, purgatory, synod, contagion, chancellor, wretchedness, convocation, archbishop (87)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| +Acad        | drawback, absence, career, lending, culture, deduction, learner, command, hearing, variance, grace, philology, warrant, solemnity (14)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | communication, thinker, sunday, earnestness, aid, amount, monday, goal, collector, intuition, graduate, engineer, friday, afternoon, decease, infirmary, street, thursday, saturday, rapidity, interpreting, clerk, expiration, builder, council, surveyor, habit, chapel, vocabulary (29)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | majesty, kingdom, accommodation, preservation, hall, interest, article, study, succession, promise, zeal, person, authority, family, advantage, design, example, care, foundation, proof, foot, assistance, treasure, concert, support, chemistry, knowledge, power, goodness, concern, manufacturer, exertion, relief, posterity, resolution, honour, carpet, manufacture, preserving, reputation, favour, name, esteem, inspection, management, prejudice (46)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | loon, jelly, room, rill (4)                                                                                                                                                                                                                                                                    | thinker, graduate, aid, rill, drawback, pretension, collector, culture, lending, elocution, preservation, absence, deficiency, furnishing, rendering, communication, cheapness, deduction, manual, engineer, teacher, room, loading, grace, repute, correspondence, enlargement, equipment, variance, handling, treasure, advance, posture, detail, demand, report, builder, vacation, condition, repair, attendance, service, access, solemnity, example, street, hearing, preserving, school, readiness, guidance, relation, navy, care, guide, saturday, management, monday, beginning, deal, content, order, arc, labour, thursday, making, taking, society, place (69)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | learner, distribution, article, hook, carpet, sunday, kingdom, steadiness, establishment, intuition, credit, foot, promise, majesty, relief, proficiency, surname, tackle, ordnance, fount, employment, ease, surveyor, growth, enjoyment, encouragement, support, algebra, vigour, student, discipline, physics, manufacture, interpreting, afternoon, tutor, excellence, fidelity, advancement, basis, command, assistance, supply, fishing, space, form, march, road, limit, trade, branch, advantage, mechanism, arithmetic, possession, philology, composition, court, rate, concern, science, compilation, syntax, board, chemistry, deliberation, study, future, pursuit, point, filling, profit, livelihood, success, observation, foundation, residence, vocabulary, delay, infirmary, perception, superiority, acuteness, rapidity, friday, discharge, infant, climbing, anniversary, resolution, gift, reputation, fineness, posterity, provision, field, industry, opportunity, teaching, chair, highness, family, mathematician, mic, council, committee, exercise, practice, power, discernment, person, level, salary, tic, time, maintenance, clerk, hospital, learning, wood, day, consideration, work, number, agility, tenderness, weaving, geography, dignity, prejudice, acquisition, benefit, mathematics, office, child, welfare, subsistence, importance, commission, favour, pretence, scholar, keeper, consent, authority, revenue, geometry, zeal, artillery, acquaintance, payment, approach, approbation, neglect, expiration, contrary, evidence, duty, doubt, goodness, proof, professor, desire, honour, homage, treasurer, proportion, neighbourhood, privilege, faculty, princess, concurrence, chapel, meeting, affection, manner, winter, analysis, knowledge, province, morning, compendium, claim, clearness, fondness, money, book, nature, inquiry, name, skill, sum, decease, quantity, wealth, esteem, necessity, reverence, friendship, exactness ... (216)                                                                       |
| +Acad+       | training, value, tact, patient, hearer, estimate, scrutiny (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | lecturer, cloth, contribution, rote, clue, depth, forte, germ, familiarity, edification, sound, zoo (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | modelling, wisdom, merit, pleasure, application, perfection, ledge, perseverance, discovery, attention (10)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | slate (1)                                                                                                                                                                                                                                                                                      | modelling, lecturer, rote, beginner, sequence, clue, scrutiny, germ, ledge, watt, discoverer, nursery, forte, collateral, tuition, theme, diploma, doc, collation, patient, vocation, expert, information, examination, aim, drudgery, certificate, drawing, foundling, term, attention, travel (32)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | hearer, seminary, value, product, sound, bio, sis, governess, carpenter, merit, edification, trout, observatory, weaver, nicety, perseverance, profession, fulness, depth, ability, diligence, discovery, infancy, wisdom, boy, pleasure, patience, harmony, leisure, maturity, childhood, familiarity, college, utility, penetration, master, degree, experience, dexterity, mason, perfection, mu, facility, delight, subject, cultivation (46)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Active      | organization, suppression, philology (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | urge, partnership, intervention, bass, promoter, concession, volunteer, fusion (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | precision, fee, filly, example, merit, preservation, blast, fad, posterity, instance, zeal, style, proof, ledge, constitution, honour, perseverance, kingdom, safety, neighbour, mutt, effect, reputation, capacity, study, ornament, attention, player, dress, performance, neatness, esteem, wisdom, assistance, mechanic, tributary, commerce, design, injury, judge, support, agency, business, mankind, chemistry, protection, stretch, equity, manufacture, knowledge, paper, elegance, animal, bulwark, union, salvation, species, deity (58)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | muff, haft, stile, peak, frill, rake, flue, jade, puss, pie, razor (11)                                                                                                                                                                                                                        | flue, ledge, lye, temperance, tributary, fare, player, directory, hare, drill, uniform, legation, frill, furnishing, urge, lure, rake, toast, pie, jade, elocution, toilet, wage, preservation, puzzle, expert, criterion, plea, mechanic, itch, stretch, partnership, preferment, fan, firm, equipment, banquet, display, paint, exit, tennis, suppression, fore, communicating, hiring, curb, intervention, joke, bulwark, avail, task, administration, dint, performer, tuition, breed, system, trump, example, promoter, strike, volunteer, saving, attention, couple, safety, whist, labour, trader, applause, pedigree, stationer, printing, fiddle, praise, access, experiment, union, correspondent, venture, judge, business, breeding, renown, wit (85)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | puss, steadiness, puff, procedure, fuel, blast, fusion, quill, object, naturalist, inducement, opponent, ingenuity, neatness, production, practitioner, razor, scratch, perseverance, tackle, kingdom, precision, proficiency, cable, instance, concession, legislator, style, talent, merit, knot, recipe, deviation, effect, caution, employment, shape, operating, ornament, profession, stroke, posterity, reputation, knave, compass, compilation, composition, workmanship, brush, imitation, dress, acquisition, neighbour, derivation, repast, credit, spear, weapon, mechanism, magnet, bow, invention, industry, constitution, excellence, equity, success, instrument, accomplishment, support, manufacture, understanding, cure, education, ability, practice, science, assistance, protection, pen, foresight, trick, propagation, heraldry, paper, adversary, munificence, sagacity, combat, honour, poker, probity, nicety, capacity, tooth, defender, lawyer, legislature, harm, exercise, bribery, attainment, pump, statesman, chemistry, recourse, principle, philology, impudence, precept, study, advancement, washing, remedy, compliance, maxim, proof, rule, flow, species, precedent, blaze, skill, injury, dexterity, mix, sanction, integrity, abuse, mankind, adherence, zeal, commerce, valour, mixing, esteem, animal, proverb, measure, honesty, mu, historian, wisdom, taste, accuracy, variety, tongue, elegance, cylinder, botany, brass, prudence, fortune, profit, cutting, learning, sex, testimony, facility, work, deity, eloquence, devil, reformation, number, definition, phrase, thumb, salvation, virtue, armour, experience, wealth, knowledge, moderation, ambition, judgment, reasoning (178)                                                                                                                                                                                                                                                                                                                                 |
| +Active+     | delegate, veto (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | application, manufacturer, machinery, series, arrangement, government, contrivance, bait, management, kite, slide (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | pattern, strip, stall, belt, fence, mule, jib, rifle, cement, runner, filter, piece (12)                                                                                                                                                                                                       | truss, strip, filter, draft, rally, mule, runner, pawn, quadrille, cement, tool, veto, rating, umpire, stud, jockey, jib, bait, annex, truck, competitor, delegate, extension, digest, scrip, contrivance, tug, team, challenge, transit, section, trot, substitute, shift, executive, mining, publishing, manual, band, guide, heating, fencing, apparatus, cost, inventor, survey, management, piece, game, maker, price, giving (52)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | distribution, establishment, hook, fob, tram, spin, formation, coalition, slide, crack, kite, file, steel, tape, baking, string, government, twist, pack, farming, bribe, bayonet, spade, sweep, gut, pin, algebra, preparation, purchase, hammer, ram, synopsis, harness, introduction, regulation, undertaking, collection, cleaning, arithmetic, machine, branch, pan, nail, husbandry, grant, ivory, trade, negotiation, plough, geometry, whip, prosecution, parliament, metal, vote, defence, publication, manure (58)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| +Ambig       | presentment (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | deputy, builder, preface (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | king, zeal, support, style, heart, government, knowledge, power (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                                                                                                                                                                                                                                | presentment, builder, navigation, peace (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | government, pe, support, baggage, child, committee, style, king, blood, gi, deputy, negotiation, shop, day, welfare, loyalty, lady, poem, alliance, treaty, work, church, heart, power, history, hill, zeal, ground, book, preface, volume, knowledge, money, plant, grief, religion (36)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| +Ambig+      | career (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | goal (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | safety, succession, accommodation, security, happiness, reason, instance, honour, effect, advantage, arrangement, perspiration, preserving (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                | knight, perspiration, safety, preserving, report, ball, haste, content (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | result, profession, effect, instance, approach, prosperity, maintenance, security, expedition, genius, advantage, honour, digging, declaration, reason, pa, happiness, confidence, cause, love (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| +Changed     | regime, drawback, loathing, canning, annoyance, retention, veto, congratulation, trait, goodwill, mania, hostility, supporter, lunacy, viewing, endowment, advert, cessation, consummation, delineation, advent, incorporation, confiscation, threat, desertion, ruler, privation, mood, boon, presentment, lending, greed, licensing, deduction, irritability, craving, auditor, sect, imbecility, disaster, semblance, symptom, lifetime, culture, detention, variance, accord, seizure, compulsion, absence, import, agitation, suppression, scrutiny, sneer, par, formula, grace, irony, exhaust, taint, manifesto, mover, piracy, destiny, delegate, ego, patient, peril, chagrin, intrigue, correction, emphasis, hearing, solemnity, currency, longevity, heroine, abbe, warning, strife (81)                                                                                                                                               | feud, childe, longing, lecturer, caring, clue, herald, serf, thinker, germ, chef, ache, seer, semi, role, rote, sig, negro, papacy, convulsion, goldsmith, dwelling, anticipation, discouragement, renunciation, cheque, forte, rite, commune, gurney, guild, derby, graduate, trash, closing, aid, enigma, thrill, budget, sob, lunatic, duo, wail, meantime, comp, debility, turkey, reprobation, bey, communication, sine, remonstrance, retort, mater, stoppage, sharpness, breadth, array, moralist, note, blockade, labor, urge, abstinence, harp, humiliation, peninsula, banking, collector, star, ballot, data, choosing, retribution, lodger, gateway, recitative, forum, haven, victor, dissent, labourer, embryo, transformation, ringing, partnership, engineer, gilt, basil, parentage, workhouse, lane, intervention, builder, ending, suggestion, obstruction, jubilee, volunteer, din, mourning, arbitration, demolition, marquess, reckoning, stipend, chronicle, vita, street, opus, scaffold, moor, monday, divorce, relaxation, saturday, vein, entry, tuesday, accounting, jail, wednesday, gazette, duel, promoter, thursday, quid (127)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | modelling, ledge, tank, squirrel, darling, helm, cairn, ice, pail, tent, trail, demon, preservation, tarn, reptile, log, mechanic, tributary, litter, wick, elf, jar, shadow, seam, jet, dolphin, perspiration, rug, toilette, stretch, player, magistrate, wile, jug, sherry, profusion, paw, affair, consequence, companion, leopard, greyhound, treasure, rib, mote, doctor, bait, frame, safety, fragment, stranger, beaver, vegetation, bulwark, herd, occasion, example, contrivance, attention, way, balance, business, preserving, conversation, search, management, union, judge, care (69)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | frill, bouquet, tray, leek, gig, punt, divan, disc, jib, tress, kilt, mule, recess, patch, doll, strip, filter, cider, runner, prow, rill, gear, jade, flue, rake, vest, cone, pie, thistle, zinc, manufactory, cement, peat, chine, rift, diver, bath, monkey, hat, cottage, room, piece (42) |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Collective  | hostility, career, piracy, presentment, consummation, outcry, value (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | era, goal, cloth, democracy, chairman, coming, sunday, monday, regeneration, street, bond, thursday, week, friday, tuesday, tinge, communication, cross, saturday, highway, jubilee, cycle, penny, wednesday, array, wreck, walker, beach, suggestion, corps (30)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | promise, majesty, article, resolution, union (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | coon (1)                                                                                                                                                                                                                                                                                       | hostility, presentment, consummation, piracy, tweed, jubilee, spark, stock, blackness, array, suggestion, communication, press, rout, street, ruby, scotch, demand, union, saturday, bank, monday, information, yacht, thursday, beginning, wednesday, tuesday, place (29)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | democracy, regeneration, scalp, atlas, gauntlet, tinge, froth, spreading, promise, chairman, sunday, board, cycle, cable, article, cross, cotton, cup, quarter, penny, emerald, primrose, chasm, flag, coming, subsidy, granite, week, field, majesty, growth, value, resistance, cradle, ordnance, commission, mortar, tower, corps, friday, schooner, chair, revolution, mint, outcry, wreck, fleet, kiss, walker, prosperity, resolution, beach, cruise, time, treasury, muslin, ship, brigade, day, artillery, grant, declaration, ridge, dividend, month, eruption, barge, shore, anchor, dawn, horse, bay, port, evidence, crew, battle, sword, vote, boat, sky, coast, oath, river, arrival, voyage (85)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Collective+ | regime, organization, manifesto, advent, proclamation, currency, mania, conflict, stagnation, conference (10)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | league, herald, peninsula, storm, aid, contribution, tide, october, ebb, rise, convulsion, gallery, haven, budget, mall, february, tempest, hurricane, flood, september, zoo, gazette, january, council, turkey, panic, earthquake, morn, channel, fusion, demolition (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | hall, blast, spray, empire, concert, discovery, paper (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | rill, sea (2)                                                                                                                                                                                                                                                                                  | regime, budget, herald, haze, aid, rill, entirety, rime, convulsion, mania, advent, turkey, glee, mist, haven, carol, prance, sheen, chili, directory, mining, currency, racing, peninsula, manifesto, protest, china, gathering, cologne, hail, hull, demolition, gazette, advance, extension, congress, report, convention, condition, titan, sway, season, renown (43)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | blast, fusion, stagnation, proclamation, spray, foam, supremacy, rainbow, fount, reek, panic, ebb, beet, coinage, hurricane, cataract, tide, bubble, lava, whirlwind, sea, contest, continent, confederacy, boom, wave, christendom, sweep, storm, snow, land, paper, banner, council, discovery, anniversary, march, market, empire, meteor, monopoly, roar, invasion, morn, triumph, insurrection, tempest, gale, blaze, breeze, accession, glow, victory, channel, july, june, country, shipping, august, october, news, importation, meeting, september, january, february, earthquake, november, december, dominion (70)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| +Conc        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                | modelling, ledge, tank, squirrel, darling, helm, cairn, ice, pail, tent, trail, demon, preservation, tarn, reptile, log, mechanic, tributary, litter, wick, elf, jar, shadow, seam, jet, dolphin, perspiration, rug, toilette, stretch, player, magistrate, wile, jug, sherry, profusion, paw, affair, consequence, companion, leopard, greyhound, treasure, rib, mote, doctor, bait, frame, safety, fragment, stranger, beaver, vegetation, bulwark, herd, occasion, example, contrivance, attention, way, balance, business, preserving, conversation, search, management, union, judge, care (69)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | carpet, kite, herring, cob, slit, pea, flight, article, promise, blast, lath, rim, bucket, ion, oar, cove, mat, soot, rat, mirror, snail, onion, dress, figure, kingdom, shaft, spray, slide, gad, bit, ornament, wing, diadem, biscuit, stack, precision, majesty, cot, beetle, moth, neighbour, posterity, protection, appearance, merit, neatness, instance, discovery, concern, countenance, stocking, glove, party, inclination, thing, satisfaction, fringe, light, carving, hoop, security, reel, perseverance, intention, cushion, chemistry, dart, lawn, freight, glory, boa, style, wife, effect, vat, urn, foot, relief, rabbit, event, world, commerce, bowl, empire, toe, broom, comb, pleasure, speck, government, answer, equity, bed, paper, constitution, end, reputation, copy, prejudice, stalk, fluid, assistance, foundation, goodness, support, injury, honour, belly, king, advantage, notion, species, wisdom, happiness, house, idea, lustre, family, capacity, animal, glass, throne, cage, wretch, heaven, hand, study, monarch, scene, malice, title, proof, friend, father, manufacture, grease, head, reason, soul, person, favour, antiquity, mercy, distance, resolution, blessing, creature, perfection, strength, water, motion, authority, power, twine, prince, zeal, elegance, salvation, esteem, name, deity, fire, courage, argument, providence, heart, mankind, bosom, eye, meaning, knowledge, image (172)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Conc+       |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                | frill, bouquet, tray, leek, gig, punt, divan, disc, jib, tress, kilt, mule, recess, patch, doll, strip, filter, cider, runner, prow, rill, gear, jade, flue, rake, vest, cone, pie, thistle, zinc, manufactory, cement, peat, chine, rift, diver, bath, monkey, hat, cottage, room, piece (42)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | plaid, puss, rind, nymph, mug, curve, skeleton, razor, lump, bite, equipage, camera, crocodile, sea, spider, load, morsel, wreath, scroll, spring (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| +Desc        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | mockery (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | precision, world, satisfaction, promise, succession, squirrel, inclination, happiness, thing, biscuit, hall, exertion, intention, posterity, filly, wisdom, goodness, performance, mutt, glory, neighbour, way, wife, darling, hive, reason, safety, spray, pleasure, shadow, heaven, affair, ledge, bucket, soul, honour, wretch, conversation, mercy, countenance, security, father, sherry, strength, rim, prejudice, notion, companion, kingdom, idea, pea, cove, throne, modelling, paw, authority, broom, answer, stalk, consequence, accommodation, malice, blessing, dart, demon, cur, courage, ice, salvation, magistrate, mat, glimpse, figure, light, visitor, vat, snail, soot, treasure, head, reel, litter, bed, rug, esteem, heart, mechanic, log, judge, mote, toe, cot, pail, frame, equity, rat, preserving, jar, bit, stranger, jug, meaning, oar, mankind, bulwark, balance, slide, twine, speck, doctor (110)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | bouquet, pattern, screen, slate, mug, bath, curve (7)                                                                                                                                                                                                                                          | bouquet, demon, greeting, darling, ledge, squirrel, modelling, pail, jar, shadow, log, lure, sherry, paw, assemblage, yawn, bath, jug, pool, gorge, ice, litter, thinking, gutter, pane, companion, rug, mechanic, magistrate, suffering, stranger, glen, doctor, shark, conversation, physic, brute, mote, applause, affair, bulwark, trot, safety, way, treasure, consequence, frame, balance, peal, step, preserving, peace, saying, memory, judge (55)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | bang, bucket, froth, pore, cove, oar, boll, festivity, promise, rim, spray, hiss, biscuit, reel, rap, rush, pea, mug, jest, swing, scar, slide, soot, lamentation, snail, thrush, swearing, precision, mat, countenance, howl, rat, thing, wife, bit, posterity, curve, kicking, figure, ghost, brood, gust, silence, answer, dart, moment, neighbour, curl, inclination, satisfaction, kingdom, cot, hem, exultation, cleft, toe, malice, prejudice, laughter, glory, wretch, vat, mane, pardon, voice, speck, incense, world, intention, thankfulness, gallop, equity, pleasure, gliding, grandeur, smile, existence, mockery, goodness, wisdom, dignity, shame, happiness, blessing, light, passion, curse, heaven, idea, soul, honour, stalk, weeping, reproach, prayer, reason, father, security, ferocity, misery, anger, sigh, understanding, broom, temper, notion, anecdote, worship, throne, salvation, strength, courage, resurrection, behaviour, stanza, tenderness, spirit, affection, bed, pride, thirst, eternity, realm, tranquillity, heart, terror, humour, oath, flow, warmth, twine, lark, obedience, time, blaze, friendship, exercise, mercy, word, esteem, contrary, mankind, despair, authority, love, contempt, conscience, consent, god, sleep, head, freedom, bough, delight, corruption, abuse, wrath, joy, youth, compassion, hatred, breast, envy, tongue, meaning, innocence, liberty, indignation, evil, piety, guilt, truth (172)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| +Desc+       |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | sob, sink (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | blast, perspiration, helm, hand, tarn, slit, stretch, trail, cairn, wile, gad, kite, onion, bosom, cob (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | muff, railing, stall, mud, stile, peak, belt, jib, frill, gig, jelly, patch, loon, fence, punt, cone, tress, moss, haft, sitting, divan, kilt, leek, prow, tray, strip, rake, bite, cider, recess, rill, mule, hat, runner, sea, rift, coon, thistle, hip, load, rind, lump, pie (43)          | frill, gig, sob, punt, tress, rill, jib, divan, trail, patch, strip, prow, haze, leek, kilt, rake, wile, tray, runner, mule, rift, cone, cider, reverie, tarn, perspiration, thistle, stretch, helm, hum, recess, cairn, lather, chatter, pie, wink, thud, chime, mister, cub, mist, nod, moving, bight, dive, mire, hat, whisper, throng (49)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | slit, tick, hush, rind, onion, blast, cob, bite, kite, gad, gasp, rave, murmur, moan, cataract, load, lump, sea, glare, git, kit, swarm, groan, rage, kiss, breath, frown, slumber, hand, bosom (30)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Freq        | tact, exaggeration, sadness, annoyance, missionary, utterance, organization, career, conflict, fervour, estimate, irritation, temperament, ruler, mania, endowment, sphere, emotion, absence, allusion, exposition, failure, arrest, recognition, carelessness, delineation, hostility, semblance, disciple, epoch, destiny, boon, mood, suppression, value, martyrdom, agitation, strife, courtesy, peril, rebuke, warning, massacre, comment, enthusiast, element, phraseology, advent, verdict, heroine, consummation, consecration, hearing, musician, weariness, supporter, emphasis, lifetime, churchman, license, attitude, interpreter, reformer, correction (64)                                                                                                                                                                                                                                                                          | burst, anticipation, sympathy, bitterness, elder, gallery, distrust, park, chairman, storm, lecturer, earnestness, intellect, household, singer, closing, encore, gladness, echo, street, workhouse, width, herald, peninsula, era, explanation, democracy, labourer, derby, movement, comprehension, contact, star, believer, evening, terrace, denial, barrister, region, lamb, ache, saint, cathedral, semi, choosing, gateway, concession, flash, goldsmith, victor, hesitation, comrade, tempest, october, agony, ending, dwelling, rector, array, meantime, chill, harp, theory, atmosphere, butler, scepticism, intervention, thrill, justification, creed, extinction, manufacturing, communication, habit, sink, depth, calmness, biographer, lunatic, disregard, link, conception, painter, revolt, vocabulary, collector, explosion, banking, lace, abbey, mademoiselle, mockery, data, beach, stability, condemnation, vestry, exclusion, summary, brick, penny, ride, contribution, bond, chapel, frankness, gilt, suggestion, engineer, parentage, panic, ringing, reliance, rapidity, moor, blindness, chronicle, genesis, humiliation, termination, walker, wreck, bloodshed, familiarity, priory, consistency, heiress, willingness, basil, pang, parish, sales (132)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | promise, hand, world, shadow, satisfaction, light, interest, inspection, material, agency, fragment, companion, chemistry, sketch, perfection, arrangement, glimpse, machinery, family, visitor, satin, father, mirror, stretch, protection, name, neighbour, carpet, manufacturer, junction, shaft, stranger, station, search, ornament, discovery, blessing, meaning, conversation, manufacture, way, spray, doctor, knowledge, heart, head, friend, management, party, nave, demon (51)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | sitting, cottage, pattern, recess, screen, manufactory, skeleton, strip, rifle, vest (10)                                                                                                                                                                                                      | meantime, mania, annoyance, recording, boon, closing, desert, bachelor, host, endowment, anticipation, glee, lecturer, herald, semi, semblance, uniform, ruler, lunatic, substitute, mystery, exterior, occupation, data, strip, shadow, stem, fellowship, absence, suffering, manufactory, advent, wanderer, fairy, domain, proximity, harp, mist, hostility, demon, derby, choosing, novelty, masonry, pilgrim, recess, goldsmith, stretch, glen, agitation, victor, exclamation, delineation, mood, antiquarian, ache, fete, remnant, vehicle, star, banking, gathering, destiny, collector, dismay, vest, notoriety, peril, nuisance, supporter, cottage, stranger, tailor, bidding, nook, consummation, correspondence, passenger, furnishing, doctor, nursery, companion, possessor, tablet, humiliation, gateway, dwelling, deposit, sport, pantomime, sufferer, flora, lifetime, greeting, speculation, stud, peculiarity, toilet, banquet, moat, workhouse, extension, counting, lust, churchyard, stake, workman, cabinet, racing, watering, thrill, chivalry, job, suppression, joke, sergeant, employer, vision, tiger, intervention, vocation, theme, stamp, parentage, refreshment, array, assemblage, sensation, margin, peasant, champagne, legion, guise, asylum, suggestion, speed, staff, guest, display, fragment, emphasis, bust, scotch, saving, leader, manual, waiter, score, detail, firm, rescue, transition, whisper, basil, periodical, waste, rarity, adventure, moving, peninsula, bible, china, band, warning, gilt, tuition, couple, hunting, attire, guidance, usefulness, teacher, opening, engineer, cheapness, apparition, shelf, section, task, street, trumpet, strike, ballad, ringing, performer, base, spectator, chance, generation, messiah, nonsense, protest, chronicle, communication, moor, squire, executive, advance, toast, villa ... (268) | sailing, arrest, distribution, skeleton, sanctuary, exaggeration, carelessness, mound, naturalist, identity, cleanliness, courtesy, establishment, formation, combination, store, royalty, shaft, detection, democracy, mirror, reviewer, carpet, opponent, illusion, romance, hush, hospitality, twist, indication, lamb, ghost, atlas, concession, stability, comrade, creed, extinction, supremacy, casting, reformer, fervour, sting, biographer, illumination, suffrage, chest, activity, damsel, elder, modification, suicide, trout, rush, specimen, dash, drama, guy, border, hook, crack, impression, fulness, fashion, echo, diary, proprietor, peep, porch, mill, silence, tract, owner, promise, fiction, steel, removal, sadness, predecessor, flash, catastrophe, enthusiast, chairman, martyrdom, salmon, theory, profession, painter, sphere, production, comfort, glare, chill, region, limit, comprehension, baroness, driver, license, monster, sweep, believer, belle, conclusion, distrust, pastor, laughter, fishing, churchman, mademoiselle, recreation, perspective, injunction, governess, observer, consciousness, devotion, irritation, scoundrel, cup, distinction, personage, massacre, pilgrimage, repetition, atmosphere, park, dialect, cotton, value, explanation, gladness, vestry, cash, mechanism, richness, dining, area, farewell, rebuke, cradle, daylight, bolt, lace, trap, basis, stroke, willingness, likeness, restoration, bloodshed, shock, disgust, contract, landlord, reply, denial, comment, verge, reference, selling, lamp, model, destruction, student, unity, architecture, panic, housekeeper, gulf, turf, robin, granite, pig, revival, burst, accumulation, politics, flower, delusion, barrister, excellence, condemnation, rumour, shape, risk, ingenuity, device, preparation, exclusion, similarity, boundary, assassination, sculpture, side, murmur, consistency, vase, inspiration, farming ... (753)                                                                                                       |
| +Freq+       | tone, case (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | cloth, cross, amount, list, dinner, rise, feeling, madame, sound, castle, fact, square, note (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | dress, instance, style, business, example, effect, study, strength, idea, series, glass, attention, soul, heaven, scene, appearance, figure, application, distance, union (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | spring (1)                                                                                                                                                                                                                                                                                     | rest, capital, note, opera, race, step, sort, system, drawing, information, school, cost, press, class, music, attention, mark, example, season, business, memory, saying, stage, union, labour, theatre (26)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | sound, result, cross, object, square, space, moment, dress, table, action, artist, existence, instance, extent, effect, style, figure, boy, rose, grave, sight, education, picture, period, chair, scene, spot, collection, field, expression, act, possession, castle, progress, science, voice, practice, library, appearance, glass, summer, stone, village, university, property, dinner, literature, idea, presence, professor, hair, rate, supply, colour, garden, meeting, study, palace, window, spring, road, sun, century, wood, door, strength, taste, poet, reading, respect, difficulty, college, building, soul, description, story, iron, sister, song, influence, heaven, importance, wall, mother, distance, health, food, passage, husband, evidence, girl, experience, poetry, principle, silver, tree (96)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| +Human       | irritation, tact, drawback, delineation, musician, organization, deception, delirium, privation, layman, mood, culture, impetuosity, confiscation, penalty, distraction, disciple, sphere, axiom, longevity, annoyance, abbe, irritability, follower, tone, ego, lending, interruption, threat, churchman, agitation, emphasis, heroine, imbecility, malady (35)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | league, communication, burst, feud, brandy, vita, encore, mount, clue, blight, indigestion, trash, terrace, cloth, goo, peninsula, lamb, intellect, banking, meal, singer, barrister, valentine, ride, reckoning, moralist, goldsmith, tinge, mutton, era, echo, allowance, coquette, pint, entry, haven, mall, beer, shoemaker, nerve, region, relaxation, blockade, lodger, debility, contribution, lunatic, bombardment, gateway, tea, widower, rapidity, kinsman, mile, wheat, parentage, termination, atmosphere, channel, clerk, contact, wail, biographer, rabbi, promoter, meantime, epidemic, constable, wreck (69)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | animal (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                | drawback, feud, watering, clue, privation, bachelor, circus, delineation, banking, culture, fete, mood, host, publican, lunatic, ven, meantime, flora, moralist, threat, annoyance, wail, watt, confiscation, debility, glee, goldsmith, communication, evacuation, lodger, trash, imbecility, fife, ditty, vagabond, lending, blockade, dismay, parentage, irritability, haven, honor, plight, agitation, sensation, adventurer, reckoning, pilgrim, flock, pet, gateway, emphasis, attendant, longevity, hermit, relaxation, peninsula, seaman, tithe, ex, toll, abbe, subscriber, peasant, vita, ego, entry, traveller, breed, promoter, monsieur, leader, villa, heroine, situation, season, toil, making, part, taking (80)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | deception, layman, follower, yeoman, blight, leech, goo, valentine, shoemaker, lamb, drunkard, indigestion, irritation, governess, mill, churchman, practitioner, delirium, seminary, tinge, bravo, bombardment, malignity, axiom, scoundrel, legislator, sou, distraction, shopkeeper, dupe, suburb, biographer, tenement, rye, coquette, brood, continent, fishing, burst, allowance, echo, canal, vizier, barrister, region, farming, solace, impetuosity, mum, sphere, dealer, ride, housekeeper, rendezvous, author, rent, brandy, atmosphere, cad, owner, rascal, cat, penalty, epidemic, widower, beggar, fish, matron, rabbi, nerve, trading, ascent, pastor, kinsman, interruption, consternation, mount, boy, extent, cape, malady, hypocrite, ransom, boundary, executioner, constable, disciple, globe, cruise, tobacco, tea, escort, moon, rapidity, farm, beer, equator, animal, cure, vigour, running, village, elector, prospect, liquor, impostor, vale, channel, wreck, frenchman, clerk, land, sailor, alacrity, frontier, governor, malt, farmer, oats, fortress, planting, perception, summer, soldier, englishman, mutton, duchy, bay, woe, mile, captain, sheep, army, rivulet, importation, meadow, orbit, proportion, entrance, harbour, rain, intercourse, winter, pint, month, wheat, garrison, payment, pasture, translator, weather, river, coast (153)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Human+      | temperament, career, ruler, enthusiast, desertion, supporter, training, trait, patient, sneer, rebuke, mover, delegate (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | childe, serf, negro, seer, commandant, liar, comrade, sympathy, bey, episode, victor, role, retribution, infatuation, basil (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | mechanic (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |                                                                                                                                                                                                                                                                                                | serf, childe, trait, supporter, negro, seer, ruler, desertion, role, notoriety, wanderer, spy, bey, workman, informant, mechanic, notary, dancer, virtuoso, prefect, prodigy, patient, mover, retribution, potentate, victor, omen, scribe, basil, elocution, tailor, vagrant, sire, sneer, delegate, spaniard, clown, lad, joke, beau, idiot, trader, generation, mien (44)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | liar, enthusiast, subaltern, comrade, damsel, huntsman, guy, nun, laird, rebuke, dragoon, planter, belle, mourner, innkeeper, artist, episode, guile, proprietor, commandant, inhabitant, pirate, infatuation, butcher, commentator, admiral, rabble, merchant, stupidity (29)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| +Indiv       | delineation, abbe, disciple, phraseology, irony, comment (6)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | singer, niece, childe, melancholy, indigestion, widower, kinsman, apothecary, heiress, lodger, unhappiness, incarnation, shoemaker, misconduct, calculation, baseness, brevity (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | wife, cur, magistrate, stranger, elegance, sherry, fragment, deity, demon, rat, reptile, antiquity, arrangement, scene, image (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | belt, stile, pattern, patch (4)                                                                                                                                                                                                                                                                | demon, childe, legation, exterior, drill, reverie, recording, job, inoculation, patch, simile, delineation, stranger, lodger, reptile, sherry, notary, physic, receptacle, curb, feu, mystery, labyrinth, magistrate, waiter, irony, sergeant, detail, abbe, ex, groom, fragment, rebus, trace, idiot, traveller, retirement, curiosity (38)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | reviewer, foreman, governess, indigestion, shoemaker, dialogue, rat, cleanliness, imitation, client, livery, widower, pique, admiration, valet, melancholy, oracle, pigeon, kinsman, phraseology, crook, partner, narration, secrecy, solace, madman, comment, multiplicity, niece, ferocity, grave, unhappiness, acuteness, vermin, wife, innkeeper, devotion, fitness, similarity, allegory, rectitude, arithmetic, garret, misconduct, meditation, steward, swine, brevity, bailiff, disciple, incarnation, invocation, page, amusement, calculation, regularity, dissipation, lawyer, source, solitude, repast, miracle, anecdote, painting, coachman, description, apothecary, commentator, butcher, divinity, metaphor, mistress, variety, adversary, conjecture, frontispiece, rhyme, escort, baseness, elegance, grandfather, deity, scene, mildness, depravity, consolation, remark, offspring, widow, understanding, button, philosopher, impostor, variation, antiquity, meekness, keeper, scripture, baptism, eucharist, adultery, sagacity, devil, poet, simplicity, apartment, madness, discourse, adoration, idleness, youth, prayer, facility, duration, arabic, anatomy, heiress, authenticity, ridicule, accuracy, sister, analogy, cousin, tradition, humility, grammar, reasoning, image (128)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| +Indiv+      | annoyance, mood, ego, carelessness, temperament, hearer, case, irritability, allusion, courtesy (10)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | inexperience, choosing, caring, levity, forte, impertinence, rudeness, intuition, reliance, mademoiselle, serf, evasion, implication, courtship, biographer, basil, grocer, infirmity, painter (19)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | fee, ornament, visitor, mechanic, neatness, companion, figure, family, father, safety, doctor, animal, player, wretch, conversation, judge, herd (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | gig, muff, mule, cottage, sitting, monkey, nymph (7)                                                                                                                                                                                                                                           | gig, mule, spy, bachelor, annoyance, pew, monitor, serf, employer, mood, attic, mechanic, choosing, caring, wink, freak, monkey, vehicle, doctor, expert, forte, aide, prefect, player, novice, matrimony, persona, attendant, basil, adult, exclamation, companion, privacy, gent, milliner, scribe, miser, irritability, stab, cottage, gallantry, couple, nursing, abbot, brute, guest, apparition, cell, counting, sire, whim, tribe, dint, pet, hermit, card, attire, sufferer, ego, nook, performer, patrician, prodigal, posture, whist, squire, asylum, tiger, gesture, situation, herd, transaction, engraver, choice, conversation, safety, supper, angel, victim, defending, judge (81)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | detection, hearer, carelessness, damsel, tenement, dupe, impropriety, courtesy, ass, irregularity, neatness, oversight, shopkeeper, suicide, ease, noun, mademoiselle, huntsman, impertinence, ghost, intuition, biographer, drunkard, comfort, figure, soliloquy, baroness, mourner, implication, inexperience, practitioner, ornament, scoundrel, courtship, evasion, courtier, kitty, singularity, gardener, mouse, orphan, nymph, protector, solicitude, dick, convenience, reverend, politician, personage, hypocrite, arbour, reproof, observation, caprice, dolly, matron, painter, indulgence, reliance, astonishment, bridegroom, rudeness, artist, abbess, friar, gravity, propriety, parent, caution, diffidence, coolness, nicety, meanness, sparrow, timidity, negligence, quickness, omission, epithet, restraint, animal, disposition, infirmity, condescension, chastity, parlour, inhabitant, waking, ostentation, disguise, father, wretch, levity, decorum, elegy, minute, alacrity, deceit, temper, grocer, delicacy, family, stupidity, context, kind, affection, decency, ancestor, kitchen, apology, flattery, civility, gentleness, angle, aperture, affectation, modesty, syllable, interval, prudence, firmness, indolence (122)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| +Interp      | inaccuracy, penalty, endowment, lord, culture, consecration, hostility, appropriation, martyrdom, estimate, disaster, irritation, loathing, boon, command, accomplice, longevity, indictment, exposition, suppression, art, ego, manifesto, trial, champion, correction, delegate (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | childe, labourer, cloth, league, park, regeneration, burglary, cross, breadth, tuesday, monday, goal, peninsula, lane, papacy, sunday, gallery, infusion, dwelling, width, cathedral, herald, derby, alias, warren, street, vita, array, wednesday, recorder, cycle, friday, lace, decease, perversion, chef, thursday, goldsmith, era, negro, focus, builder, gentlewoman, fortnight, semi, sympathy, niece, chairman, strand, tinge, marquess, star, explosion, rite, mount, unwillingness, argent, renunciation, plaintiff, banker, feud, countess, forte, adjournment, judaism, chaplain, viscount, terrace, bologna, coroner, infirmary, vicar, priesthood, mall, poultry, gaol, liver, february, encore, butler, saturday, chapel, gilt, exchequer, walker, county, week, note, earldom, brick, finale, mutton, reliance, baker, gurney, longing, coffee, dean, app, parcel, inconsistency, opus, sig, horseback, singer, region, abbey, guild, goo, movement, abstinence, dinner, circuit, castle, graduate, october, reservation, highway, choir, tenant, rector, intemperance, bombardment, collector, aurora, denial, fox, thinker, turkey, corps, bond, biographer, partnership, debility, accounting, square, ire, afternoon, implication, bey, inquest, tide, comp, gateway, ballot, exclusion, duo, deputy, provost, role, stability, intuition, list, january, custody, evasion, explanation, parentage, shoemaker, faithfulness, link, fact, channel, thesis, apothecary, organist, obstruction, convent, limitation, vault, candle, scaffold, preface, arbitration, inferiority, parish, promoter, lunatic, apprentice, ward, forum, summary, september, misrepresentation, rider, willingness, priory, sales, evening, widower, embryo, conception, grocer, lease, saint, concession, rapidity, deanery, wheat, constable ... (207)                                                                                                                                                                     | machinery (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                                | childe, chef, semi, herald, boon, graduate, rite, endowment, negro, dwelling, role, papacy, derby, goldsmith, feud, culture, watt, turkey, guild, flora, gurney, hostility, loathing, disaster, forte, meantime, informant, array, debility, note, sig, longing, star, breadth, collector, lunatic, bey, renunciation, thinker, comp, toilet, gilt, builder, forum, parentage, workman, operator, peninsula, collation, domain, embryo, temp, ballot, terra, jockey, milliner, deficit, duo, labourer, china, collateral, gateway, warehouse, arbitration, partnership, transit, suppression, lane, street, chili, delegate, marquess, hunting, deficiency, longevity, deposition, racing, repair, abstinence, beau, promotion, obstruction, exchange, suggestion, correction, manifesto, tavern, stationer, legion, promoter, monday, fencing, maker, certificate, ego, saturday, tuesday, vita, wednesday, school, villa, scaffold, resort, opus, yacht, engraver, thursday, relation, cost, subscriber, part, temple, accounting, coach (114)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | regeneration, appropriation, cycle, cross, thesis, lace, infusion, liver, fox, inaccuracy, tinge, pendant, stability, unwillingness, shoemaker, tenant, formation, sunday, atlas, martyrdom, modification, cotton, park, biographer, gentlewoman, goo, warren, bombardment, baker, governess, fresco, focus, reservation, art, accomplice, solving, bio, square, willingness, rider, adjournment, coal, banker, chairman, observatory, canal, staple, implication, intuition, perversion, region, corps, champion, device, interment, explosion, field, judaism, consecration, cathedral, exclusion, butler, penalty, carpenter, steel, indictment, matter, recorder, frock, inferiority, aerial, sales, workmanship, walker, hemisphere, evasion, concession, aurora, coroner, freehold, diamond, limitation, predecessor, friday, tine, inquest, consistency, irritation, fertility, brick, gaol, advocate, similarity, extent, niece, ordnance, chaplain, plaintiff, lens, weaver, suburb, reliance, command, denial, board, vault, space, oil, enamel, ire, bologna, compilation, infirmary, globe, week, boy, ore, tower, viscount, faithfulness, priesthood, dean, jeweller, rebuilding, misrepresentation, afternoon, constable, abbey, spot, arsenal, chamber, argent, embroidery, app, explanation, modem, chapel, altitude, gown, granite, castle, apprentice, boundary, silk, division, mortar, flag, lord, mason, reservoir, market, accident, earldom, widower, trial, tide, brigade, colour, vicar, candle, palace, road, surgeon, lease, tic, cape, exposition, number, quarter, pair, conception, apothecary, iron, intemperance, view, citadel, building, channel, mount, warden, county, summary, inconsistency, mansion, apparel, imperfection, countess, provost, zone, chalk, composition, custody, maintenance, digging, marshal, parish, deanery, eclipse, deputy, proportion ... (291)                                                                                                                                                               |
| +Interp+     | transportation, tact, canning, organization, regime, drawback, lack, delineation, retention, recognition, congratulation, lunacy, ruler, archdeacon, sphere, missionary, incorporation, consummation, career, deception, training, craving, trait, licensing, supporter, veto, viewing, value, dealing, lending, variance, critique, synod, absence, stress, allusion, confiscation, exaggeration, musician, enthusiast, magnitude, case, compulsion, cessation, absolution, exhaust, deduction, warrant, impediment, verdict, par, sect, failure, advert, desertion, piracy, symptom, goodwill, learner, mania, destiny, archbishop, auditor, canon, prosecutor, disciple, arrest, conference, intrigue, abatement, proclamation, heroine, scrutiny, follower, currency, element, reluctance, philology, mover (79)                                                                                                                               | lecturer, diocese, communication, clue, amount, aid, bishopric, executor, acquittal, contribution, blockade, anticipation, engineer, tariff, account, commissioner, manufacturing, urge, council, valuation, germ, expansion, hardship, freeman, painter, banking, fusion, volunteer, calculation, divorce, index, budget, envoy, surveyor, data, gazette, demolition (37)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                | regime, canning, drawback, retention, trait, lecturer, clue, veto, consummation, supporter, aid, congratulation, viewing, lunacy, advert, delineation, incorporation, ruler, germ, licensing, sect, blockade, lending, mania, goodwill, confiscation, budget, scrutiny, absence, communication, urge, anticipation, transfer, mining, engineer, craving, variance, data, occupation, par, banking, desertion, media, rarity, auditor, cessation, mover, piracy, beginner, deduction, intrigue, award, destiny, tuition, exhaust, symptom, periodical, demolition, compulsion, capital, digest, cabinet, cheapness, volunteer, currency, inventor, divorce, survey, report, gazette, experiment, aim, heroine, order (74)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | deception, fusion, cleanliness, learner, valuation, enthusiast, arrest, follower, tariff, sphere, bishopric, perspective, freeman, exaggeration, value, magnitude, expansion, painter, commissioner, acquittal, committee, critique, richness, impediment, proclamation, diocese, purpose, stress, surveyor, verdict, viceroy, prosecutor, archdeacon, astronomer, hardship, architecture, student, council, canon, executor, farming, calculation, abatement, sultan, ratio, artist, metropolis, cardinal, journal, commission, synopsis, reluctance, philology, work, monument, supply, absolution, principality, embassy, disciple, account, synod, pursuance, forgery, archbishop, candidate, church, index, description, symmetry, affinity, epitome (72)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| +Judg        | exaggeration, privation, penalty, organization, mover (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | epilogue, levity, tea, prologue, mediocrity, unwillingness, punch, penny, parody, brevity, biographer, banking (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | fee, occasion, performance, affair, thing, consequence, material, intention, design, pleasure, mutt, reason, instance, example, business, paper, fad, argument, creature, wretch, arrangement, judge, malice, mankind (24)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | rake, muff, piece (3)                                                                                                                                                                                                                                                                          | rake, privation, lye, mover, substitute, temperance, fare, workman, dole, toast, harlequin, banking, consequence, brute, affair, joke, banquet, pantomime, nonsense, task, press, occasion, piece, clown, sage, sort, class, bard, cost, choice, bargain, example, venture, business, readiness, diet, content, part, correspondent, praise, wit, place, judge, giving, theatre (45)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | epilogue, exaggeration, whit, puppy, prologue, bully, cake, mediocrity, impropriety, punch, bravo, specimen, biographer, talent, drama, unwillingness, matter, propensity, preference, malignity, fashion, frolic, parody, company, comfort, jest, amusement, paper, author, thing, brevity, farce, credit, tinsel, goose, penny, gaiety, instance, purpose, lyric, epigram, scruple, liquor, temptation, penalty, kind, profession, tea, refinement, matron, disdain, bot, rule, comedy, court, partiality, indulgence, desire, gentleman, committee, taste, artist, pleasure, foreigner, humour, disposition, manner, intention, conceit, villain, chastity, morality, undertaking, madam, maxim, odium, custom, dissipation, loss, work, admirer, scorn, leisure, pudding, precedent, couplet, devil, entertainment, woman, writing, passion, invective, amendment, sex, malice, fault, wretch, rhyme, pride, goddess, creature, reason, food, levity, iniquity, attempt, subject, versification, man, crime, patience, people, word, vanity, misfortune, refusal, song, servant, pound, freedom, temper, proverb, trouble, absurdity, shame, delicacy, wickedness, mirth, reader, stupidity, treason, body, opinion, modesty, money, sentiment, winter, judgment, principle, mankind, proportion, punishment, meat, writer, skill, obligation, severity, nobleman, truth, book, censure, innocence, vinegar, affectation, simplicity, offence, army, indignation, injustice, argument (160)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Judg+       | sacrilege, greed, intrigue, champion (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | trash, mutton, impertinence, blasphemy, coquette (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | honour (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                | greed, opium, trash, sooth, fop, commoner, chivalry, gallantry, intrigue, beau, adventurer, peasant, fitter (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | drunkard, impertinence, scandal, self, sacrilege, coquette, knave, patron, champion, slander, fool, ostentation, sloth, infidel, rogue, adulation, coward, libel, blasphemy, modem, drinking, mutton, ridicule, disgrace, abuse, licentiousness, flattery, hypocrisy, decency, fraud, impostor, honour, probity, meanness, insult, impiety, arrogance, extravagance, folly, virtue, beef, contempt, calumny, indolence, decorum, liberty, ingratitude, honesty, tyrant, moderation, generosity, prudence, cruelty (53)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| +Man         | sneer, veto, utterance, disaster, strife, courtesy, intrigue, recoil, attitude (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | goal, echo, mutton, partnership, recorder, burglary, vein, region, chairman, habit, ear, apoplexy, provost, deputy, inquest, liar, flash, parody, clerk, harp, collector, embryo, earldom, familiarity, dollar, pang (26)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | mechanic, dart, wretch, snail, beetle, neighbour, broom, bed, bowl, eye, cairn, sherry, paw (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | cider, nymph, spider, bite, filter (5)                                                                                                                                                                                                                                                         | veto, filter, cider, mechanic, hock, fare, strait, employer, pilgrim, desert, glee, temperance, spark, partnership, toast, remnant, miser, job, fop, gait, disaster, champagne, tribune, embryo, lad, cairn, gallantry, banquet, vagrant, harp, sham, brand, theme, collector, sherry, couple, bull, intrigue, vein, pageant, sweat, whist, novice, paint, adventure, paw, sneer, guest, flute, pencil, victim, tuition, seaman, pulse, chance, transport, grove, mien, throng, sage, swimming, antagonist, inventor, toil, retirement, breeding, walking, sway, aim, vacation, chaise, wit, strife (73)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | bitch, naturalist, opponent, rack, legislator, thrush, recoil, liar, twist, vineyard, duck, snail, muck, glare, twain, dollar, courtesy, beetle, echo, dupe, bite, dart, spider, beggar, dragoon, toad, swing, apoplexy, pinch, perch, frolic, hospitality, moan, subaltern, mouse, plume, snake, heron, plumage, bubble, mast, region, grave, nymph, astronomer, invader, guile, knot, foam, parody, string, flash, palate, boom, sweep, bowl, falcon, relish, ear, rigging, propensity, mate, crest, groan, ape, chop, torch, bailiff, laughter, inquest, chairman, pang, chin, merriment, hypocrite, turtle, spleen, brake, repast, englishman, thirst, complacency, sparrow, leap, dialect, wretch, jeweller, mathematician, crowd, beak, recorder, neighbour, goose, forest, thicket, commentator, boy, drinking, devastation, smile, owl, orator, rudder, rash, stature, brutality, manhood, humour, snow, box, meteor, jaw, flesh, canvas, visage, scorn, admiration, derision, strain, adversary, critic, fate, triumph, fortune, bow, keeper, footman, dust, deck, adjutant, stuff, bird, tribute, rage, cow, curse, throat, shoe, fowl, cat, forehead, shield, mountain, blaze, breath, fish, dwarf, provost, flame, frenzy, harvest, mutton, oak, member, accident, hedge, tooth, disdain, habitation, bag, bed, deputy, pride, barn, wound, mayor, pair, clerk, candidate, couplet, delight, familiarity, grandfather, pile, broom, liquor, hunger, brook, temper, bough, frost, button, nose, crew, hell, gleam, historian, pine, cask, age, youth, worm, sky, acquaintance, estate, noise, gale, trunk, gloom, attorney ... (237)                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Man+        | canning, musician, ruler, supporter, career, archdeacon, conflict, follower, layman, enthusiast, disciple, delegate, reformer, indictment, churchman, champion, mover (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | labourer, lecturer, thinker, goldsmith, negro, builder, comrade, banker, rider, bass, shoemaker, bey, moralist, fox, horseback, apprentice, biographer, barrister, derby, slaughter, grocer, painter, ride, gaol, punch, highway, victor, tenant, serf, kinsman, apothecary, organist, duel, din, bearer, moor, nerve, graduate, broker, promoter, jail, constable, walker, firing (44)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | blast, helm, reptile, player, herd, posterity, monarch, kite, greyhound, leopard, rabbit, bait, belly, trail, cob, log, animal, beaver (18)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | gig, mule, punt, rifle, monkey, runner, belt, razor, crocodile, hip (10)                                                                                                                                                                                                                       | canning, mule, gig, lecturer, supporter, thinker, punt, draft, watt, baton, publican, goldsmith, runner, reptile, umpire, negro, ruler, workman, trail, player, bey, monkey, circus, miner, truck, hare, jockey, greyhound, celt, serf, moralist, vehicle, wink, spy, helm, host, team, log, graduate, cricket, virtuoso, discoverer, stud, passenger, waiter, otter, tool, racing, sport, competitor, staff, tailor, tally, prefect, brute, neigh, leopard, bait, delegate, gull, watchman, joke, mister, mover, flock, derby, expert, shark, repute, hound, victor, stab, sergeant, builder, crow, elk, trump, herd, stake, axe, clown, connoisseur, breed, operator, race, curb, tiger, band, trumpet, loading, foe, trader, fife, hunting, labourer, wage, trot, antiquarian, knight, leader, adventurer, fugitive, fist, dilemma, vagabond, speed, dutchman, tribe, fiddle, traveller, steeple, moor, duel, rout, citizen, din, beaver, spectator, groom, strike, promoter, bard, guide, galley, fencing, clan, game, tavern, jail, halt, maker, felon, mark, engraver, luck, renown, quack, coach (138)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | follower, hobby, shoemaker, reformer, fray, cob, leech, razor, file, rider, bang, fox, pun, buckle, boar, grin, hook, huntsman, tape, quill, comrade, flint, bully, stag, kite, puppy, mill, driver, bravo, ass, horseman, shopkeeper, scoundrel, foreman, laird, layman, bowling, biographer, drunkard, crack, blast, caliph, butt, yeoman, kicking, lance, buck, gap, booth, bolt, rod, crocodile, horn, rascal, tenant, trout, howl, ranger, turf, trophy, crook, kennel, madman, bearer, carnage, lion, enthusiast, mallet, politician, rabbit, dealer, painter, robber, punch, spade, stroke, practitioner, sexton, dash, churchman, livery, landlord, banker, scratch, hawk, tackle, parade, cock, weaver, monster, bayonet, predecessor, chaff, chariot, mace, deer, shepherd, pigeon, pig, observatory, assassination, hunter, pack, courtier, dun, gunner, carpenter, hack, hog, personage, tanner, elephant, drum, brace, ox, boot, frog, champion, butcher, vermin, executioner, contest, jumping, carver, bush, woodcock, student, belly, apprentice, quarry, coachman, rogue, fork, scourge, volley, brush, swearing, cutter, caravan, gang, gallop, pheasant, spear, shirt, innkeeper, dragon, skull, ram, hammer, barrister, corporal, fishing, trick, swine, statesman, indictment, coward, slaughter, countryman, dick, poker, combat, limb, knave, trap, kinsman, beard, bacon, mane, steel, constable, roar, whirlwind, fool, heel, ride, frenchman, weapon, warrior, lightning, valet, benefactor, preacher, whip, rabble, ensign, killing, track, eagle, villain, barrel, cane, gaol, camel, cardigan, dagger, booty, patron, client, broker ... (319)                                                                                                                                                                                                                                                                                                                                                                                                  |
| +Neg         | threat, disaster, inaccuracy, allusion, coldness, massacre, semblance, disorder, intrigue, irony, warning, visitation, outcry, chancellor (14)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | burglary, perversion, discouragement, inconsistency, rudeness, papacy, levity, disapprobation, refutation, vehemence, unhappiness, tempest, disregard, narrowness, impertinence, panic, reprobation, agony, slaughter, explanation, meddling, distrust, parody, impeachment, explosion, antipathy, apoplexy, chill, earthquake, unwillingness, termination, violence, atheism, infirmity, justification, wail, obstruction, din, hurricane, penitence, stoppage, robbery (42)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | inclination, security, thing, succession, safety, neighbour, throne, reason, bed, intention, monarch, posterity, king, occasion, idea, glory, precision, mercy, concern, prejudice, argument, notion, instance, performance, protection, preservation, magistrate, fragment, exertion, equity, judge, event, soul, empire, answer, resolution, end, soot, zeal, satisfaction, meaning, providence, rat, constitution, stretch, ledge, scene, government, frame, countenance, agency, bulwark, business, discovery, paw, wife, conversation, father, log, cage, litter, creature, doctor, courage, heart, mankind, elf, moth, balance, bosom (70)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | stile, belt, muff, moss, jelly, peak, slate, bite (8)                                                                                                                                                                                                                                          | threat, disaster, stab, discouragement, incredulity, papacy, heft, mystery, wail, log, ledge, stoppage, semblance, reprobation, gutter, blackness, stretch, plea, magistrate, doctor, speculation, bulwark, puzzle, paw, litter, intrigue, elf, preservation, prevalence, protest, system, fragment, juncture, shark, miser, felon, gesture, irony, safety, nonsense, administration, frame, adventurer, obstruction, din, warning, step, tiger, riot, memory, defending, bourbon, conversation, corpse, balance, occasion, crisis, state, intent, judge, transaction, correspondent, trace, business, separation, curiosity, saying (67)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | coalition, howl, bang, gust, accumulation, opponent, rack, kicking, progeny, scar, impropriety, meddling, carnage, refutation, disapprobation, propensity, rat, froth, charge, panic, rush, perversion, narrowness, shipwreck, ghost, soot, bite, drowning, jest, inaccuracy, contention, impertinence, lament, massacre, removal, commonwealth, unhappiness, contest, caprice, parody, merriment, venom, pique, vermin, apoplexy, maze, deviation, existence, procedure, vigilance, fable, answer, repetition, thing, equity, unwillingness, neighbour, protection, chill, groan, detriment, inclination, government, whirlwind, visitation, moth, unanimity, inability, circumstance, rudeness, indulgence, exultation, concern, obscurity, infidel, laughter, prejudice, uncertainty, confederacy, instance, humanity, posterity, countenance, extent, impeachment, jargon, caution, event, omission, reproach, conclusion, impudence, trap, antipathy, inconsistency, rule, regency, revolution, disadvantage, consciousness, disorder, recovery, scene, explanation, clemency, dissipation, security, conflagration, slaughter, interposition, affront, mandate, expediency, timidity, detestation, robber, idea, bed, heap, prevention, toad, precision, explosion, imitation, glory, deed, king, distrust, rag, passion, impunity, diminution, monarch, pardon, despotism, resolution, meanness, fallacy, throne, hurricane, end, disregard, infirmity, ague, rabble, artifice, tendency, amazement, dagger, discovery, intention, disguise, scruple, coldness, paganism, levity, felony, revenue, murder, temper, plague, redress, partiality, idleness, neglect, wife, defect, abhorrence, adversity, bill, reason, englishman, legislature, tumult, scorn, decay, fright, killing, volley, adherence, beak, recourse, commotion, empire, behaviour, indifference, presumption, pestilence, torrent, presence, inequality, tempest, notion, incapacity, arrogance, crying, dungeon, cage, belief, contradiction ... (338)                                           |
| +Neg+        | sacrilege, desertion, deception, annoyance, privation, penalty, strife, confiscation, detention, relapse, peril, conflict, wretchedness, loathing, menace, weariness, failure, accomplice, imbecility, mortification, recoil, mania, contagion, interruption, malady, carelessness, chagrin (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | retaliation, bitterness, remonstrance, blasphemy, feud, intoxication, disaffection, fanaticism, revolt, banishment, evasion, misrepresentation, mockery, rashness, retribution, insensibility, intemperance, prohibition, anarchy, schism, sob, trash, infatuation, bloodshed, pang, blindness, hardship, blight, epidemic, humiliation, misconduct, baseness, perplexity, sink, demolition (35)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | blast, consequence, affair, reptile, perspiration, wretch, injury, trail, shadow, demon, herd, malice, dart, cairn, snail, slit, cur, beetle, kite, spray, leopard (21)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | mud, railing, fence, mule, patch, strip, haft, rake, jib, crocodile, load, monkey (12)                                                                                                                                                                                                         | demon, mule, sob, trail, feud, strip, reptile, spy, barrier, annoyance, retribution, desertion, privation, rake, patch, humiliation, pallor, imbecility, mania, freak, clutch, suffering, yawn, plight, remonstrance, peril, loathing, lust, confiscation, curb, perspiration, poisoning, mist, inoculation, sufferer, cairn, shadow, dismay, brute, stain, disuse, sham, detention, apparition, exclamation, jib, base, rout, mire, monkey, trash, suspension, consequence, nuisance, herd, servitude, chagrin, leopard, affair, vagrant, crow, exit, dilemma, demolition, foe, partition, strike, victim, whisper, idiot, filth, cell, snare, labyrinth, rescue, antagonist, waste, strife, haste, hazard, dissolution (81)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | retaliation, deception, detection, recoil, fanatic, fatality, puff, scoundrel, annihilation, bully, menace, blast, irregularity, drunkard, bloodshed, hush, hiss, grin, glare, dupe, kite, suicide, rust, malignity, sting, sacrilege, temerity, slit, complication, delusion, gasp, evasion, blight, monster, cramp, scourge, madman, soldiery, invader, rascal, disgust, scandal, pang, misrepresentation, swearing, silence, catastrophe, prohibition, crocodile, epidemic, lamentation, indignity, blunder, barbarism, destruction, snail, rap, assassination, mask, carelessness, accomplice, snake, knave, insensibility, beetle, moan, fanaticism, puppy, witchcraft, temptation, disaffection, blindness, mortification, murmur, moment, misconduct, heron, devastation, revolt, indiscretion, intoxication, relapse, load, dart, mockery, swarm, slander, sloth, adversary, brutality, trick, bubble, imposition, profligacy, complaint, malady, infatuation, brood, hardship, disturbance, mistake, spray, weariness, alarm, frown, frenzy, offender, derision, blasphemy, ferocity, spleen, grievance, penalty, chaos, hypocrite, malice, desolation, wretch, libel, infidelity, rage, bitterness, licentiousness, combat, baseness, deceit, bribery, curse, iniquity, wretchedness, depravity, negligence, imposture, rashness, coward, confinement, beggar, invective, discord, fool, pretext, crush, expulsion, distress, sedition, fate, contagion, falsehood, doom, rash, deformity, intemperance, disgrace, banishment, odium, injury, imprudence, vexation, traitor, superstition, encounter, obstinacy, popery, terror, heretic, faction, owl, shame, darkness, forgery, torment, schism, insolence, interruption, risk, dust, calumny, drunkenness, barbarity, anger, wickedness, hell, dread, usurper, tear, disobedience, immorality, usurpation, roar, disappointment, confusion, delay, madness, misery, invasion, fraud, animosity, displeasure, abuse, anarchy ... (287)                                                                           |
| +Object      | canning, semblance, archbishop (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | justification, anthem, imperfection, condemnation (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | precision, succession, preservation, dress, merit, injury, constitution, design, example, rim, pea, cushion, biscuit, ice, carpet, inclination, glory, fringe, copy, protection, jar, filly, application, inspection, antiquity, shaft, urn, stack, ion, mercy, proof, kingdom, comb, study, esteem, instance, toilette, elf, grease, title, neatness, blessing, rib, hoop, wife, reputation, carving, leopard, slide, power, jet, mankind, twine, conduit, speck (55)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | stile, peak, kilt, mud, divan, patch, cone, peat, cement, room, razor (11)                                                                                                                                                                                                                     | jar, ledger, canning, whore, pallor, divan, kilt, toilette, film, simile, patentee, jet, ration, raft, peat, ice, grit, paving, preservation, semblance, pith, patch, presentation, loop, sequence, rib, stem, cement, deposition, elf, codex, cone, toast, dispersion, poisoning, antiquarian, fellowship, lust, leopard, applause, enlargement, deposit, peculiarity, function, information, administration, margin, writ, band, couple, stamp, cabinet, sentence, examination, rape, room, bench, character, memory, printing, example, relation, choice, praise, saying, state, society, order (68)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | tape, carpet, detection, shaft, rim, coke, pea, fringe, slide, razor, ingenuity, dress, froth, stack, swing, porch, hoop, drawer, blemish, epistle, unity, precision, ion, urn, kettle, imitation, glare, chamber, physics, curl, sheet, foam, neatness, fabric, ham, merit, gasp, biscuit, interment, cushion, condemnation, string, protection, copy, prism, deviation, steel, bedding, holiness, vase, rigging, pardon, communion, kingdom, education, conformity, speck, granite, injury, fibre, carving, comb, side, plumage, humanity, constitution, porcelain, chair, fineness, drapery, imperfection, request, fidelity, harmony, form, commission, instance, picture, brush, muslin, treasury, cornice, shape, pope, inclination, college, blush, page, crucifixion, antiquity, treasurer, torch, glory, spear, wife, reputation, church, kiss, crush, sacrifice, sanction, divinity, testimony, evidence, pile, library, university, blessing, presence, grease, blanket, flame, vote, utility, trough, righteousness, twine, chimney, integrity, invective, rebellion, assembly, secretary, apartment, justification, belief, canopy, president, purse, worship, damask, nail, bead, writing, study, declaration, pulp, title, estate, taper, essence, symbol, sermon, oration, defence, freedom, power, tenderness, mankind, blaze, text, esteem, manuscript, reign, importance, ceremony, prayer, sword, paragraph, delicacy, resurrection, proof, reading, feast, velvet, letter, christianity, perfume, veneration, moderation, gospel, archbishop, faith, lordship, conscience, charity, mercy, sacrament, oath, conspiracy, learning, doctrine, girdle, covenant, mother, censure, thigh, death, religion, justice, sincerity, piety, quotation, guilt, obligation (195)                                                                                                                                                                                                                                                                                    |
| +Object+     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | rise (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | material, blast, spray, modelling, soot, ornament, ledge, tank, profusion, slit, rug, hand, litter, cairn, fragment, paper, bed, diadem, satin, seam, machinery, cage, contrivance, dart, paw, nave, mat, trail, log, preserving, glove, lustre, lath, mirror, onion, jug (36)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | jelly, bouquet, frill, belt, pattern, tray, jib, screen, slate, recess, disc, cider, strip, haft, scroll, filter, moss, rifle, wreath, fence, flue, zinc, gear, thistle, pie, tress, hip, morsel, loft, rill, mug, skeleton (32)                                                               | filter, bouquet, disc, frill, flue, zinc, strip, tray, tank, modelling, cider, masonry, haze, recess, ledge, stew, jib, thistle, pie, reef, font, gear, truss, fan, rill, tress, trail, clutch, litter, toilet, cairn, seam, sewer, rug, thud, log, sheen, mist, pool, shelf, suspension, packet, lotion, gorge, jug, paw, contrivance, core, profusion, pane, statuary, shark, gloss, fragment, base, chancel, mosaic, preserving (58)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | slit, soot, packing, distribution, bang, blast, spray, diadem, leak, skeleton, lath, accumulation, hilt, onion, tinsel, gust, pad, towel, rook, curling, ornament, mirror, action, mug, worsted, rack, paper, mat, closet, rig, grate, morsel, scroll, glove, lard, incense, portmanteau, dart, crape, embroidery, sleeve, uniformity, scarf, dagger, causeway, wreath, cage, dome, poker, softness, lap, bed, lustre, vellum, symmetry, radiance, collar, pistol, brocade, tapestry, hand, pocket, pillow, helmet, apron (65)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| +Pain        | irritation, transportation, semblance, lunacy, rebuke, absence, detention, destiny, reluctance, absolution, intrigue, emotion, seizure, massacre, hindrance, outcry, bane, indictment, bankruptcy, defiance, attitude, chancellor (22)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | burglary, dissent, distrust, rudeness, ache, disapprobation, banishment, narrowness, levity, celibacy, debility, vehemence, disregard, panic, decease, reprobation, workhouse, refutation, explanation, misconduct, epidemic, impertinence, justification, wail, priesthood, inferiority, antipathy, apoplexy, echo, reconciliation, penitence, scaffold, urge, exemption, earthquake, assent, jail, termination, infallibility, robbery (40)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | preservation, security, inclination, safety, posterity, mercy, fee, happiness, throne, wisdom, neighbour, protection, reason, hand, ledge, glory, goodness, soul, frame, concern, empire, constitution, providence, succession, wife, zeal, prejudice, commerce, satisfaction, business, exertion, thing, equity, agency, strength, monarch, idea, countenance, argument, answer, attention, father, authority, darling, heaven, friend, salvation, heart, blessing, mankind, government, instance, judge, machinery, mutt, courage, balance, example, discovery, end, resolution, family, care, bosom, event, meaning, creature, foot, foundation, dart, stretch, support, union, herd, scene (75)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | muff, belt, strip, stile, patch, moss, rake, filter, zinc (9)                                                                                                                                                                                                                                  | strip, ledge, truss, debility, wail, semblance, cess, heft, incredulity, filter, ache, zinc, darling, reprobation, preservation, lunacy, rake, plea, curb, firm, patch, dye, seizure, mystery, absence, destiny, decline, intrigue, deposit, deposition, detention, frame, lust, incline, wage, urge, deficiency, stretch, dissent, system, gesture, asylum, gutter, balance, workhouse, saving, whisper, felon, safety, corpse, administration, step, exit, miser, juncture, chance, nonsense, transition, axe, summons, attention, generation, herd, memory, pedigree, jail, scaffold, restitution, bourbon, drudgery, trace, advance, business, correspondent, neutrality, peace, demand, union, navigation, example, judge, sway, labour, curiosity, care, toil (86)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | progeny, hush, opponent, shipwreck, grin, narrowness, panic, suicide, commonwealth, disapprobation, blemish, drowning, propensity, irritation, epidemic, lament, twain, humanity, rebuke, kicking, scandal, carnage, result, extent, impropriety, celibacy, protection, impertinence, refutation, unanimity, frequency, ease, murmur, countenance, procedure, existence, reserve, deviation, distrust, detriment, apoplexy, mandate, bane, mourner, consciousness, ministry, inferiority, misconduct, caution, equity, employment, maze, posterity, answer, government, renewal, circumstance, chasm, reproof, clemency, massacre, prosperity, reluctance, fabric, object, orphan, neighbour, rule, inclination, interposition, secrecy, confederacy, concern, indulgence, wife, bubble, bankruptcy, rudeness, discipline, adversity, hindrance, echo, prejudice, expediency, brood, regulation, mediation, explanation, contest, antipathy, security, cancer, dispensation, client, legislature, foot, knave, exultation, limit, conclusion, fool, amendment, protector, dissipation, spine, affront, dart, petitioner, indictment, thing, reply, amazement, detestation, disregard, assent, executioner, wave, trading, instance, scruple, hypocrite, glory, bribe, fright, cession, crying, disguise, adherence, astonishment, constitution, trade, defiance, resistance, certainty, discovery, change, settlement, banishment, empire, felony, penitent, tendency, event, compliance, parent, impudence, voice, throne, partiality, revenue, persuasion, blush, belief, idea, passion, subjection, indifference, blessing, wisdom, property, friend, pride, satisfaction, pardon, abhorrence, temper, outcry, disposition, education, freedom, community, soul, ague, commerce, support, end, paganism, mildness, decency, defender, contradiction, scorn, reformation, presence, priesthood, forgery, source, welfare, goodness, precipice, monarch, infection, sacrifice, pestilence, dungeon, reason, visage, resolution, infancy, arrogance ... (383)                |
| +Pain+       | coldness, privation, conflict, desertion, sacrilege, annoyance, deception, threat, confiscation, penalty, loathing, checking, stagnation, inaccuracy, disaster, imbecility, peril, case, strife, wretchedness, failure, visitation, relapse, menace, warning, weariness, recoil, contagion, default, taint, mortification, malady, agitation, carelessness, disorder, interruption, correction, impediment, chagrin (39)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | discouragement, perversion, papacy, bitterness, retaliation, blasphemy, inconsistency, disaffection, blight, remonstrance, intemperance, revolt, feud, intoxication, evasion, fanaticism, mockery, insensibility, misrepresentation, rashness, unhappiness, hardship, retribution, schism, unwillingness, anarchy, blindness, pang, sob, prohibition, perplexity, meddling, humiliation, infatuation, divorce, agony, infirmity, baseness, intervention, slaughter, chill, extinction, bloodshed, deafness, indigestion, remission, tempest, violence, imperfection, obstruction, demolition, atheism, sink, condemnation (54)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | blast, consequence, reptile, shadow, injury, perspiration, wretch, bed, magistrate, helm, trail, demon, bulwark, shaft, malice, relief, slit, beetle, soot, cur (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | mud, railing, fence, haft, mule, slate, load, crocodile (8)                                                                                                                                                                                                                                    | demon, mule, sob, threat, privation, reptile, annoyance, barrier, trail, feud, imbecility, humiliation, discouragement, papacy, desertion, pallor, loathing, retribution, confiscation, plight, suffering, stab, disaster, shadow, clutch, inoculation, tool, peril, strait, poisoning, sufferer, suspension, perspiration, stain, dismay, remonstrance, coil, brute, base, yawn, servitude, helm, disuse, exclamation, stake, magistrate, bulwark, dispersion, prevalence, mire, demolition, agitation, remnant, partition, intervention, survivor, consequence, blackness, protest, reduction, sewer, mist, warning, chagrin, dilemma, stem, divorce, rescue, nuisance, filth, lung, strike, vagrant, taint, correction, rout, fist, obstruction, snare, waste, victim, idiot, burden, repair, cell, antagonist, foe, dissolution, haste, crisis, sweat, brink, defending, separation, strife, state, quack, minority, refuge (99)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | retaliation, deception, detection, recoil, blight, checking, annihilation, fatality, coalition, fanatic, scar, howl, stagnation, contention, scoundrel, irregularity, rust, blast, sting, shaft, drunkard, menace, accumulation, meddling, gust, glare, rack, verge, dupe, removal, cramp, unwillingness, bloodshed, default, sacrilege, bully, extinction, invader, visitation, temerity, complication, delusion, evasion, pang, gasp, snake, silence, slit, scourge, perversion, unhappiness, malignity, inaccuracy, misrepresentation, recovery, barbarism, dirt, catastrophe, carelessness, disgust, prohibition, indignity, blindness, madman, chill, destruction, disaffection, mask, rush, remission, rascal, insensibility, charge, deafness, crocodile, lamentation, load, indigestion, moan, monster, venom, soldiery, assassination, hardship, stroke, deed, swearing, temptation, vigilance, respite, malady, sloth, cave, fanaticism, relief, grave, taxation, soot, revolt, groan, affliction, vermin, moment, beetle, mortification, mockery, disadvantage, slander, blunder, witchcraft, brutality, indiscretion, imposition, penalty, obscurity, infirmity, disturbance, intoxication, heap, uncertainty, alarm, profligacy, whirlwind, relapse, adversary, inability, coldness, prevention, spleen, devastation, revolution, derision, infatuation, complaint, distress, contagion, desolation, deformity, reproach, baseness, intemperance, weariness, blasphemy, combat, frenzy, ferocity, interment, coward, condemnation, wretch, omission, deceit, offender, iniquity, fallacy, slaughter, infidelity, wretchedness, risk, expulsion, frown, bitterness, fate, curse, diminution, inconsistency, limb, rage, negligence, depravity, crush, rent, chaos, equilibrium, grievance, reparation, infidel, imposture, shortness, timidity, licentiousness, rashness, bribery, bill, malice, rupture, falsehood, shock, doom, dust, spear, sedition, extremity, decay, disgrace, release, defect, confinement, odium, mistake ... (412)                      |
| +Passive     | sadness, loathing, irritation, exaggeration, coldness, temperament, regime, irritability, delineation, craving, privation, penalty, archdeacon, cessation, mood, distraction, delirium, symptom, auditor, tact, contentment, emotion, epoch, import, semblance, imbecility, estimate, weariness, verdict, irony, utterance, inaccuracy, mortification, absolution, visitation, failure, lord, abbe, heroine, fervour, taint, relapse, stress, chancellor, abatement, wretchedness, archbishop, patient, outcry, chagrin, malady, digression (52)                                                                                                                                                                                                                                                                                                                                                                                                   | childe, ending, brandy, breadth, rector, thrill, feud, anticipation, tinge, calmness, countess, coming, remonstrance, vicar, chaplain, blight, scepticism, gladness, melancholy, exchequer, bitterness, goldsmith, inquest, lamb, cross, perversion, unhappiness, gallery, exaltation, chill, star, gateway, rapture, mockery, household, intellect, communication, castle, papacy, cathedral, dean, commandant, peninsula, perplexity, workhouse, elder, rudeness, valentine, bond, ringing, gaol, intoxication, infusion, comrade, inexperience, vault, meal, afternoon, brick, moor, kinsman, flood, infirmary, tide, entry, february, blasphemy, humiliation, clerk, serf, reservation, distrust, din, abbey, insensibility, madame, intuition, agony, convent, banker, haven, reckoning, chapel, allowance, amount, hurricane, impeachment, relict, explanation, priory, cheque, speech, lodger, marquess, pang, deafness, reprobation, believer, assent, infatuation, evening, horseback, channel, scaffold, flash, gazette, sharpness, blindness, levity, coquette, imperfection, moralist, zoo, beach, tile, january, account, deanery, envoy, wheat, transformation, wreck, fact, september (124)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | happiness, shadow, soul, perspiration, satisfaction, head, countenance, hand, foot, fragment, soot, distance, house, image, heart (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | bouquet, sitting, room, sea (4)                                                                                                                                                                                                                                                                | loathing, childe, regime, feud, bouquet, privation, imbecility, irritability, thrill, goldsmith, anticipation, pool, semblance, cheque, mood, delineation, humiliation, serf, auditor, sheen, clutch, mystery, incredulity, rouge, informant, nabob, cessation, shadow, suffering, symptom, remonstrance, gorge, craving, glen, star, breadth, dismay, gateway, carol, papacy, perspiration, lodger, deficiency, pet, ringing, import, irony, lizard, room, reprobation, nook, moor, sufferer, haven, moralist, sharpness, adventure, vision, bible, chagrin, mermaid, transformation, ending, workhouse, ivy, din, chancel, vacancy, troop, communication, pane, notary, lad, reckoning, shark, peninsula, taint, monsieur, whisper, beginning, gazette, zenith, ruby, patient, entry, foundling, drawing, fragment, abbe, relation, abbot, highland, corner, scaffold, heroine, summons, marquess, message, villa, memory, inn, place, chaise, descent (104)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | blight, exaggeration, tinge, thorn, lamb, effigy, valentine, guy, chasm, comrade, sanctuary, distraction, chill, irritation, fatality, sadness, profile, encampment, chest, balcony, illusion, visitation, inquest, unhappiness, cross, porch, reservation, hush, deli, delirium, arbour, exaltation, melancholy, intuition, reverend, stil, lamentation, sea, gladness, enchantment, consciousness, marsh, shock, suburb, diary, soot, infusion, chaplain, vault, inaccuracy, countenance, counsellor, mortification, gasp, dormer, penalty, scepticism, moan, pang, hardness, residue, lair, lid, coldness, exultation, elder, verdict, lilt, deafness, perversion, rent, brim, regency, singularity, weariness, exuberance, pit, garret, tower, interment, guile, flash, coming, kinsman, side, mockery, allowance, roughness, foot, speech, rector, brandy, thankfulness, complacency, insensibility, lava, barbarism, impeachment, dragoon, coquette, banker, hurricane, emerald, solace, inexperience, fervour, epitaph, vicar, surname, chamber, vet, afternoon, believer, assent, brick, gaol, castle, archipelago, blindness, tobacco, desolation, malady, dean, leer, dwarf, reality, sublimity, intoxication, calmness, archdeacon, volcano, forest, grave, rudeness, cathedral, distrust, contentment, ceiling, pilgrimage, suspicion, bitterness, abatement, tide, countess, explanation, eld, citadel, sight, rosemary, altitude, congregation, expanse, childhood, relapse, pearl, infirmary, child, governor, infatuation, gate, abbey, frontispiece, fever, continent, perplexity, table, sepulchre, imperfection, flower, chancellor, tranquillity, clerk, summer, time, draught, ridge, commandant, quarter, emotion, cabin, cellar, cliff, surprise, solitude, digression, curate, odour, amazement, house, waking, composure, penitent, satisfaction, imagery, soul, clover, uncertainty, bay, flag, stress ... (416)                                                                                                                                   |
| +Passive+    | drawback, martyrdom, annoyance, congratulation, desertion, stagnation, trait, variance, hostility, grace, carelessness, lack, solemnity, allusion, emphasis, hearing, interruption, mania, greed, scrutiny, peril, attitude, lifetime, agitation, warning (25)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | longing, ache, dwelling, decease, lip, earnestness, seer, mourning, discouragement, niece, germ, hardship, sob, liter, depth, storm, caring, episode, enigma, ebb, faithfulness, earthquake, creed, wail, gentlewoman, disapprobation, atmosphere, inconsistency, renunciation, incarnation, apoplexy, dissent, prediction, banishment, contact, condemnation, aurora, mademoiselle, retribution, basil (40)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | interest, vegetation, bed, visitor, speck (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                                | drawback, sob, longing, annoyance, germ, wail, fog, trait, greed, ache, congratulation, seer, pallor, haze, dwelling, greeting, desertion, bachelor, reverie, mist, scrutiny, trance, caring, mania, wanderer, enigma, grace, blackness, apparition, discouragement, hostility, sign, churchyard, vegetation, variance, pretension, lifetime, salutation, basil, emphasis, renunciation, mourning, retribution, stain, core, peril, awe, rest, corpse, posture, agitation, yawn, solemnity, survivor, felon, filth, lying, brink, burden, dissent, veil, warning, peace, hearing, horizon, home, angel (67)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | lip, martyrdom, stagnation, sting, carelessness, creed, liter, ghost, soliloquy, rainbow, shipwreck, hoar, nun, dirt, fount, prediction, gust, scar, episode, raven, disapprobation, apoplexy, atmosphere, hardship, gentlewoman, impression, snow, mourner, liking, verge, faithfulness, drowning, daylight, heaving, cradle, ebb, mademoiselle, hue, speck, moon, incarnation, condemnation, alarm, niece, complaint, cancer, depth, crust, dryness, respite, ague, consternation, twilight, crying, aurora, indignity, interruption, dungeon, sun, storm, precipice, presence, phenomenon, fright, disturbance, drought, accident, dignity, earthquake, decease, inconsistency, limb, elegy, night, wilderness, grandmother, widow, abyss, banishment, aunt, eagerness, bed, lark, firmament, dependence, forehead, pestilence, paganism, mantle, confession, resignation, couch, dew, lady, pillow (95)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| +Perc        | archdeacon, training, canon, detention, verdict, accord, presentment, philology, seizure, visitation, recoil (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | lecturer, rector, chant, coroner, flash, vicar, elder, highway, extinction, relict, reservation, tenant, coming, priory, limitation, meal, judaism, infirmary, list, cheque, contribution, surveyor, sound, castle, ear, forum, comprehension, earldom, lodger, din, acquiescence, brigadier, familiarity, basil (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | modelling, study, glimpse, safety, stretch, biscuit, speck, stocking, spray, leopard, preserving, shadow, satin, dart, eye, slide, countenance, relief, bulwark, tributary, diadem, lath (22)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | recess, railing, bouquet, manufactory, runner, strip, camera (7)                                                                                                                                                                                                                               | lecturer, modelling, entirety, recess, bouquet, draft, cheque, baton, font, runner, fog, manufactory, factor, stretch, presentment, tributary, cologne, presentation, strip, patentee, baud, waiter, mariner, accord, fuss, leopard, forum, cornet, codex, loading, aide, roan, suspension, labyrinth, marl, seizure, terra, lotion, lodger, troop, repository, shift, exclamation, vision, mystery, detention, shadow, rendering, crouch, basil, watchman, bulwark, manual, fellowship, teacher, minster, beck, chancel, crow, survivor, sufferer, whisper, chime, dive, hull, sandwich, din, step, tuition, tiger, safety, jersey, tonnage, navigation, preserving, inn, intro, yacht, trace, horizon (80)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | detection, glare, extinction, chip, tenant, recoil, flash, keel, maze, slide, steadiness, sound, boll, lath, quiver, rush, reservation, scent, effigy, speck, counsellor, huntsman, gasp, consciousness, lock, plantation, dun, activity, buck, diadem, prism, murmur, brim, surveyor, acquiescence, den, limitation, camera, gliding, coroner, enchantment, magnet, daylight, sexton, ordnance, elder, stocking, turret, sight, bro, spray, visitation, falcon, moan, similarity, marsh, dragoon, deviation, lens, peep, pennant, vigilance, snow, canon, relief, slope, biscuit, expectation, baronet, judaism, kin, leap, astonishment, git, dart, muslin, verdict, philology, ensign, rector, gaze, countenance, gunner, dispensation, sac, approach, breathing, ear, waking, reflection, gat, comprehension, spot, encounter, cutter, commander, promontory, twilight, thicket, slumber, tern, deer, cap, visage, kindling, glade, relict, alarm, poplar, grandfather, commencement, treasurer, infirmary, accession, sparkle, vicar, archdeacon, adjutant, study, vibration, roar, expanse, meteor, boundary, coming, smile, desolation, understanding, dawn, perception, shade, gleam, blaze, consul, noon, brigadier, cliff, priory, glitter, radiance, castle, glow, eruption, gloom, maintenance, ray, breath, successor, familiarity, window, firmament, jurisdiction, bough, crucifix, phenomenon, sky, taper, city, earldom, botany, breeze, territory, eye, cheek, welfare, quadrangle, sleep, orb, hedge, blossom, abyss, shoulder, charter, frigate, arrival, pasture, captain, analysis, battalion, pillow, death, shore, patriarch, river, uneasiness, dominion (186)                                                                                                                                                                                                                                                                                                                                                                                      |
| +Perc+       | sadness, recognition, drawback, deduction, cessation, advent, conflict, arrest, advert, utterance, consummation, case, semblance, epoch, scrutiny, exhaust, interruption, estimate, symptom (19)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | goal, clue, convulsion, thrill, era, dwelling, thinker, calmness, anticipation, gladness, distrust, seer, sob, gateway, rise, sharpness, intuition, ringing, chill, haven, lip, sink, prediction, germ, closing, blindness, star, vault, stoppage, agony, moor, entry, believer, pang, data, beach (36)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | succession, tarn, slit, cairn, visitor, chemistry, rim, wick, helm, perspiration, cove, interest, trail, rug, paw, fringe, squirrel, ledge, tank, tent, junction, jar, dolphin, stack, seam, shaft, cot, inspection, cushion, rib, companion, fragment, reel, log, nave (35)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | jib, disc, belt, patch, screen, tress, prow, gig, sitting, punt, diver, skeleton, sea, filter, rift (15)                                                                                                                                                                                       | disc, advert, sob, haze, greeting, advent, clue, trail, inlet, convulsion, drawback, jar, diver, tress, thinker, pallor, raft, coil, jib, sheen, barrier, germ, film, wick, scrutiny, prow, seer, clutch, cairn, stair, span, stoppage, tarn, pool, proximity, tank, dwelling, gig, sequence, patch, reef, thrill, punt, mist, consummation, gorge, sharpness, packet, ledge, blackness, reverie, anticipation, filter, lizard, glen, thud, deduction, masonry, cessation, flaw, seam, car, scan, semblance, paw, closing, rug, pane, tent, helm, hum, apparition, gateway, trance, grit, salutation, recording, log, shark, perspiration, rib, dolphin, stem, dispersion, rift, wink, ringing, loop, fern, lighting, haven, dismay, nod, scat, data, squirrel, bight, vacancy, churchyard, yawn, nook, enlargement, stain, media, star, toll, symptom, sewer, exhaust, moor, hug, halt, deposit, ruby, stoop, naming, companion, shelf, peal, guidance, corpse, awe, brig, fragment, summons, access, bank, entry, concourse, state (130)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | bang, arrest, tick, sailing, gauge, slit, fatality, skeleton, cove, gap, encampment, scar, indication, horseman, lip, swing, hush, foam, profile, rim, shaft, cataract, prick, gust, porch, intuition, perspective, concealment, curling, illumination, prediction, hilt, leak, chasm, lair, rap, fringe, complication, mound, accumulation, drawer, chill, sadness, lull, curl, rook, hiss, illusion, creek, lag, twist, rainbow, stack, reel, toot, raven, observatory, isthmus, bolt, recovery, frequency, gulf, gladness, ether, ark, distrust, blindness, cot, piazza, rigging, celerity, cleft, portal, bias, twig, quay, boom, heaving, lilt, wand, emerald, astronomer, brine, swarm, grate, pang, sea, vault, spine, schooner, tackle, chemistry, perch, dial, jumping, rudder, growth, landing, electricity, hue, brake, synopsis, scarf, catastrophe, believer, cabin, student, lightning, attainment, lava, beech, interruption, wharf, physics, canoe, elm, crest, bridle, tension, parapet, pack, caravan, interposition, continuation, calmness, inspiration, quickness, coolness, chin, barge, cushion, chart, sod, consternation, pilot, volley, plank, settlement, ridge, pier, skull, whale, tower, contraction, discharge, cough, groan, ceiling, gate, frontier, canopy, inundation, cedar, shower, beach, meadow, serenity, pond, presence, languor, wrist, interview, anatomy, respiration, agony, jaw, shot, waistcoat, map, revelation, ship, astronomy, guard (173)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| +Pleasure    | drawback, lack, fluency, delineation, goodwill, musician, consecration, culture, art (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | childe, tuesday, cloth, street, lane, breadth, goo, infusion, mount, afternoon, poultry, era, chairman, pint, meal, baker, goal, horseback, lamb, width, strand, derby, week, vita, mall, opus, brandy, october, cycle, mutton, builder, corps, mademoiselle, marquess, haven, gateway, rote, ride, commandant, zoo, engineer, september, alias, dwelling, list, sig, warren, bologna, castle, bond, mile, comp, aid, edification, beer, january, gazette, walker, penny, square, moor, candle, tonic, dollar (64)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | biscuit, toilette, stocking, cot, bit (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | divan, pie, room, manufactory (4)                                                                                                                                                                                                                                                              | childe, entirety, drawback, divan, goodwill, watt, largo, font, toilette, gelding, dais, pie, fern, rote, glee, quadrille, tome, derby, culture, forme, delineation, roan, curry, connoisseur, tweed, equipment, furnishing, sig, span, greeting, cologne, stock, pet, fife, doc, warehouse, viol, manufactory, haven, comp, breadth, aid, gateway, dwelling, piper, masque, engineer, glen, assemblage, cornet, duct, larder, van, gathering, terra, burg, virtuoso, corpus, steeple, packet, builder, street, mermaid, waiter, commendation, bar, statuary, lighting, lane, vole, jam, row, birthday, lad, room, marquess, moor, nap, ball, beau, tuesday, inn, gazette, opus, ivy, stationer, swimming, birch, monsieur, term, vita, reception, coach (93)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | rowing, goo, hook, mill, coo, lamb, seller, baker, encampment, bio, vert, infusion, tine, kettle, trill, sib, fur, cycle, salmon, yarn, yeoman, mademoiselle, canary, millet, loo, dollar, bout, art, oyster, dram, corps, peep, fiction, fluency, chairman, fir, ax, tun, frolic, afternoon, recreation, nut, tin, biscuit, weaver, bit, cock, ham, booth, bro, tilt, ilk, jeweller, mare, edification, chi, space, square, lyric, porch, prism, ger, tart, woodcock, richness, rum, soup, mountain, consecration, filling, plat, warren, tot, lens, tar, boy, cot, week, gal, freehold, tackle, proficiency, pan, cap, carpenter, creek, basket, ride, suburb, stocking, composition, tobacco, wand, hunter, sim, repast, tad, field, ferry, oil, schooner, penny, eld, gi, cruise, cam, beech, poplar, wood, muslin, yin, colour, talent, tee, commemoration, diamond, quay, barge, fleet, tit, frock, bologna, nit, wharf, woodland, oat, village, mount, castle, bay, amphitheatre, brass, butter, tic, walker, pewter, pump, tab, reservoir, bunch, rivulet, ide, noon, elm, morning, brandy, commandant, road, sparkle, lake, ton, mason, damask, shop, narrative, bee, amber, leisure, frontispiece, candle, mu, spoon, cape, ant, paste, ivory, clover, telescope, merchant, pepper, beer, horseback, brigade, gun, entertainment, june, gate, mutton, anchor, variety, plate, captain, pint, boat, river, mosque, mile, facility, shore, digging, drinking, hill, july, pasture, prose, pair, september, harbour, coronet, rice ... (225)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| +Pleasure+   | tact, recognition, lending, critique, enthusiast, exposition, accord, trait, presentment, par, learner, advent (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | encore, monday, sunday, singer, wednesday, lecturer, friday, park, thursday, goldsmith, finale, chef, contribution, saturday, tea, dinner, gentlewoman, duo, coffee, solo, herald, chant, evening, forte, harp, thinker, star, oratorio, aurora, gilt, gallery, guild, terrace, nightingale, punch, jubilee, madame, role, anthem, choosing, vein, graduate, mater (43)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | hall, sherry, satin (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | bouquet, cider, piece (3)                                                                                                                                                                                                                                                                      | chef, bouquet, fete, lecturer, goldsmith, watering, role, cider, carol, lending, harp, trait, presentment, herald, guild, thinker, forte, duo, circus, cricket, sheen, spa, graduate, hock, ditty, prelude, accord, advent, baton, award, mater, gem, sprinkling, jig, stud, tennis, milliner, homo, presentation, sherry, star, champagne, par, gilt, capital, jubilee, nursery, dancer, advertiser, nook, choosing, racing, repute, delivery, drawing, salutation, tuition, housewife, fellowship, lute, cheapness, whist, scotch, refreshment, adventure, tavern, saturday, teacher, ruby, villa, ballad, periodical, exchange, monday, lily, knack, card, yacht, wednesday, circle, vein, spending, patrician, thursday, hunting, highland, vacation, season, piece, intro, information, supper, arc (93)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | holly, hobby, learner, pastry, spice, sunday, enthusiast, illumination, sip, dining, deli, peacock, pendant, atlas, pavilion, gentlewoman, rainbow, plum, naturalist, combination, trout, pic, bowling, punch, primrose, park, fishing, motto, arbour, cake, pantheon, nightingale, aurora, belle, staple, tea, friday, rendezvous, company, critique, chocolate, rosemary, emerald, evening, parable, porcelain, lemon, comedian, cup, author, cherry, compass, pearl, hawthorn, dairy, summer, flower, fountain, garland, inspiration, gardener, cookery, exposition, parlour, journal, almond, vase, dinner, abundance, synopsis, anecdote, ale, meadow, venison, flavour, octavo, claret, cream, wedding, dish, comedy, honey, orchard, blossom, firmament, library, silver, reading, retinue, extract, book, quarto, volume, wine, essay (95)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| +Plural      | supporter, trait, hearer, bane, follower, layman, delegate, disciple, auditor, deduction (10)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | renunciation, infusion, rote, app, comrade, adjournment, burglary, epilogue, mile, brigadier, dit, broker, contribution, promoter, plaintiff, grocer, jail, bid, dollar, flood (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | fee, hive, fad, kite, pea, king, visitor, fragment, stack, concern, vat, mote, pail, wing, prejudice, cushion, kingdom, reptile, ion, exertion, security, lath, majesty, bulwark, cur, stocking, freight, glove, rib, treasure, boa, wick, moth, prince, wile, cot, oar, hoop (38)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | belt, stall, stile, haft, cone, prow, patch, filter, rill, tress, mud, rind, rake, doll, rift, loft, chine, flue, nymph, equipage, runner, bite (22)                                                                                                                                           | filter, rill, font, pail, supporter, trait, rote, fern, raft, rental, tuft, flue, prow, tome, rime, tress, cess, renunciation, rift, fife, rake, itch, strait, employer, lure, lass, stair, gene, spa, clutch, lizard, cone, pawn, patch, reptile, celt, ration, passenger, allure, liege, rib, nabob, delegate, doll, playhouse, wile, scat, fore, runner, competitor, chine, rout, fop, pane, stance, mote, effusion, wort, gent, auditor, bulwark, survivor, wick, dancer, dispatch, workman, beginner, detail, physic, truce, ex, dive, fragment, cation, gem, peculiarity, shelf, wage, rape, kid, halt, tiling, performer, deduction, guest, usage, evacuation, chime, tribe, relation, fitter, hazard, promoter, bar, ottoman, elk, bookseller, function, pant, treasure, foe, jail, repair, peace, intent, navigation, denomination, reward, arc, content, navy (111)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | coo, rind, kite, hearer, fiat, epilogue, ting, char, leak, follower, prop, lath, pun, yarn, deli, fray, fen, bout, talc, imp, specimen, fice, toot, comrade, judgement, opponent, mourner, phi, layman, chip, stack, infusion, pea, loo, bane, pate, donation, blemish, sou, wing, guile, mace, recovery, pap, oar, dud, cad, pill, bib, tort, curl, lag, fart, rant, mercury, gel, dice, propensity, cert, fib, dollar, fem, caw, rut, ref, cud, invader, teen, freight, quill, jeweller, windmill, adjournment, resource, ger, grotto, boa, ion, lam, rum, mead, legislator, gram, dun, lab, glove, pic, boot, moth, fur, bite, tab, circumstance, vat, bra, gat, con, equipage, ax, fishery, confederacy, rad, kingdom, nymph, rupture, infection, dit, poplar, stocking, rem, sot, defect, concern, woodcock, hoop, kin, amp, app, deed, moo, tincture, tau, bail, cue, sedition, cam, ferment, cushion, coin, teat, augmentation, prejudice, bot, dial, ram, cot, mic, gal, broker, dad, flu, affront, adulation, computation, wave, ban, deceit, lens, bid, brigadier, fleece, shoe, king, yen, topic, scruple, ewe, cop, shrub, consumer, ore, tart, phenomenon, malt, sleeve, rag, dock, gi, ox, majesty, rigour, garment, admirer, agent, sap, inequality, child, disciple, elector, parent, ace, felony, purchaser, envy, pretence, mile, pa, eta, column, prince, pe, inundation, plaintiff, maw, highness, flank, contrary, marquis, security, variation ... (230)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| +Plural+     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | lip (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | event, foot (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | lip, leaf, flower, foot, event, tear, member, tooth, year, word (10)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Polit       | irritation, confiscation, irritability, organization, hostility, taint, allusion, delirium, threat, seizure, sneer, conflict, desertion, mania, coldness, rebuke, menace, case, transportation, synod, privation, piracy, emphasis, ego, proclamation, bane, symptom, consecration, peril, manifesto, abbe, advent, indictment, magnitude, delegate, sect, outcry, digression, chagrin (39)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | infusion, encore, league, semi, prediction, bloodshed, bombardment, parody, episode, vindication, banishment, creed, regeneration, scaffold, dissent, acquittal, panic, arbitration, epidemic, coquette, anarchy, abstinence, democracy, inconsistency, misrepresentation, acre, prologue, sob, impeachment, slaughter, envoy, viscount, flash, ballot, parcel, quid, impertinence, coffee, misconduct, jubilee, fox, moor, chill, apoplexy (44)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | filly, dress, broom, spray, reptile, arrangement, vegetation, agency, stalk, machinery, stack, glass, idea, fragment, soul, bulwark, bowl, bosom, satin, sherry, stranger (21)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | jib, bite, stile, wreath, chine, thistle, piece, spider (8)                                                                                                                                                                                                                                    | semi, mania, threat, confiscation, draft, seizure, assemblage, sob, jib, irritability, quadrille, hostility, reptile, desertion, advent, evacuation, jockey, privation, tribune, baud, harlequin, thistle, masonry, wanderer, arbitration, tablet, reef, sect, inoculation, taint, exterior, incredulity, dais, spectator, transit, vegetation, fist, emphasis, gelding, piracy, necklace, mess, pallor, peril, chine, fairy, fan, apparition, freak, opera, delegate, rating, ballot, masque, chagrin, temp, jubilee, playhouse, sensation, job, dutchman, pane, sneer, symptom, gesture, bulwark, applause, whim, salutation, axe, squire, memorandum, brute, abstinence, sherry, margin, abbot, truce, scaffold, miser, bull, stranger, gaming, abode, manifesto, moor, china, dissent, race, couple, ego, felon, system, steeple, pledge, attire, dissolution, fragment, statuary, abbe, adieu, tale, domino, victim, retirement, rebus, sway, piece, quid, horizon, advertisement, dancing (112)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | coalition, bloodshed, prediction, puff, menace, claimant, bombardment, creed, bite, invader, grin, infusion, drunkard, fiat, gaiety, regeneration, drug, enchantment, parody, panic, soliloquy, tape, irritation, merriment, belle, democracy, hiss, maple, brim, identity, fox, epidemic, spray, soldiery, stack, demonstration, rascal, hermitage, bane, delirium, bubble, prologue, pantheon, rebuke, impropriety, indemnity, thrush, proclamation, finery, episode, operating, dress, bolt, damsel, motto, coquette, dash, fable, area, bier, interlude, propensity, devastation, supremacy, politics, republican, drama, clemency, cock, drowning, farewell, abbess, impertinence, flash, resistance, magnitude, spleen, cup, pit, cession, instigation, farce, reply, misrepresentation, booty, mandate, bark, spider, dwarf, courtier, shape, lyric, bottle, heroism, viceroy, armada, impeachment, executioner, emerald, laughter, acquittal, pheasant, plume, plumage, witchcraft, chill, misconduct, exuberance, bribe, fabric, rabble, apoplexy, hue, dialogue, sanctity, indictment, causeway, epigram, abdication, masquerade, delusion, infection, hypocrite, expulsion, sultan, poppy, buff, accent, bowl, appellation, odour, insurrection, existence, soil, ram, chaos, fleece, destruction, traitor, elephant, knot, protector, slaughter, faction, collection, murder, coachman, mane, frontispiece, broom, song, consecration, digression, dance, hemisphere, jacket, monopoly, invocation, sheet, dissipation, coldness, wedding, quay, tragedy, muslin, usurper, falsehood, deceit, rag, owl, brocade, amphitheatre, stanza, swine, drinking, propriety, oration, symbol, chocolate, hell, emblem, banishment, rage, moan, marquis, paradise, clime, sun, eruption, venison, meteor, sulphur, banner, sugar, butterfly, couplet, stalk, blanket, acre, visage ... (355)                                                                                                                                                                                |
| +Polit+      | canning, loathing, regime, ruler, retention, veto, disaster, accomplice, strife, stagnation, semblance, consummation, incorporation, attitude, intrigue, sacrilege, agitation, compulsion, suppression, massacre, wretchedness, mover, destiny, appropriation, imbecility, disorder, goodwill, contagion, champion, recoil (30)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | feud, retaliation, burglary, disaffection, remonstrance, convulsion, bitterness, intoxication, papacy, blasphemy, burst, kinsman, explosion, blight, serf, fanaticism, concession, evasion, intervention, budget, peninsula, retribution, termination, hurricane, negro, turkey, stoppage, mockery, reprobation, intemperance, obstruction, humiliation, gateway, commune, tempest, baseness, robbery, highway, partnership, duel, hierarchy, condemnation, argent, comrade, disapprobation, earthquake, vehemence, infatuation, rashness, stability, meddling (51)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | blast, demon, wretch, scene (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | belt, strip (2)                                                                                                                                                                                                                                                                                | demon, feud, canning, regime, legation, budget, veto, loathing, stoppage, convulsion, spy, retention, serf, commune, rally, turkey, ruler, strait, disaster, deficit, fete, opium, humiliation, mover, brand, plight, retribution, consummation, semblance, confederation, remnant, louse, strip, blackness, papacy, negro, sham, reprobation, goodwill, agitation, remonstrance, imbecility, toast, intrigue, incorporation, guise, intervention, chivalry, bidding, rout, stab, deposition, exit, gateway, tartar, prefect, inauguration, obstruction, gallantry, nuisance, suppression, destiny, partnership, coup, shift, troth, stake, duel, banquet, fugitive, compulsion, transaction, peninsula, salute, pageant, tribe, tiger, adventurer, challenge, omen, antagonist, trump, citizen, fraternity, peasant, foe, throng, strife, peal, clan, crisis, separation (92)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | retaliation, fray, bravo, stagnation, fanatic, festivity, malignity, blight, carnage, scoundrel, concession, opponent, senator, planter, assassination, recoil, glare, howl, border, monster, accomplice, suicide, catastrophe, appropriation, blast, rack, annihilation, robber, disaffection, gauntlet, bully, suffrage, foam, stability, evasion, whirlwind, lamentation, scourge, ruling, capitol, gust, comrade, kinsman, swarm, mask, massacre, sacrilege, explosion, hurricane, barbarism, meddling, exultation, conflagration, dragon, swearing, champion, intoxication, condemnation, vermin, fanaticism, ban, hospitality, indiscretion, respite, madman, deed, portal, bride, sweep, chariot, heretic, pavilion, trophy, hierarchy, dungeon, disapprobation, pirate, frenzy, slander, worsted, knave, ferocity, scarf, mockery, rose, baseness, disorder, bitterness, scene, curse, bridegroom, volcano, burst, uproar, gang, blasphemy, torch, detestation, doom, adversary, invective, defender, ferment, libel, derision, iniquity, contagion, brutality, wretchedness, prospect, sack, patriotism, cardigan, blaze, smoke, dagger, ermine, intemperance, warrior, infatuation, meanness, promontory, stratagem, barbarity, cap, combat, rash, wretch, patriot, pestilence, liquor, villain, shield, vow, disguise, grenadier, adultery, pride, calumny, rule, shame, anger, murderer, cloud, frown, gale, tempest, arrogance, argent, mansion, marchioness, resignation, weapon, madness, hypocrisy, imprudence, hanging, nobleman, drunkenness, rampart, pillar, flame, prelate, wickedness, gloom, rashness, velvet, plague, wrath, eagle, cloak, excommunication, pressure, bigotry, earthquake, ingratitude, robe, impiety, idolatry, thunder, vehemence, armour, coronet, death, imputation, azure, horror, helmet, neck (189)                                                                                                                                                                                                                           |
| +Pos         | drawback, delineation, culture, goodwill, fluency, congratulation, sphere, exposition, supporter, art, contentment, grace, fervour, estimate, churchman (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | cloth, tuesday, sunday, monday, lane, goal, cross, friday, wednesday, chant, park, intellect, thinker, pint, infusion, thursday, street, mount, league, goldsmith, fortnight, coming, width, cycle, tea, labourer, builder, focus, afternoon, october, strand, baker, saturday, coffee, childe, dwelling, lamb, amount, star, derby, mall, meal, gallery, evening, faithfulness, terrace, brandy, horseback, parcel, app, aurora, haven, marquess, vita, sig, poultry, anticipation, earnestness, mile, rote, negro, lace, eon, finale, seer, banker, sympathy, expiration, corps, butler, opus, september, circuit, bass, link, dinner, mutton, bologna, manufacturing, expansion, warren, ride, germ, cheque, january, walker, week, square, believer, county, mademoiselle, liter, penny, depth, commandant, widower, willingness, edification, dollar, lodger, vein, castle, shoemaker, zoo, candle, acre, beach, chapel, bid, grocer, embryo, conception, punch, atmosphere, beer, universality, priory, morn, convent, nightingale, parish, tile (122)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | water, glass, sherry, design, satin, cot, stocking, perfection (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |                                                                                                                                                                                                                                                                                                | drawback, goodwill, watt, thinker, goldsmith, culture, cheque, font, supporter, watering, forme, childe, germ, delineation, congratulation, rote, furnishing, sig, seer, dwelling, negro, derby, hock, sheen, span, fellowship, star, viol, teacher, tome, pew, gelding, grace, packet, roan, lass, milliner, haven, warehouse, anticipation, builder, dais, spa, donor, embryo, sherry, tweed, fife, ditty, van, sprinkling, lodger, diploma, charm, connoisseur, adult, staff, bar, lane, usefulness, elocution, twin, street, loan, nook, ruby, possessor, liege, commendation, vocation, stationer, lad, corpus, music, pet, hull, giver, marquess, promotion, report, villa, card, saturday, labourer, monday, wednesday, china, vein, hoard, tuesday, bust, thursday, lily, service, yacht, opus, season, inn, class, jersey, eminence, vita, renown, travel, home, coach, part (107)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | mill, holly, cycle, rowing, lamb, pendant, cross, sunday, hook, baker, bio, sib, infusion, dollar, dining, art, tine, fur, peacock, yeoman, seller, trout, shoemaker, sip, rainbow, willingness, auto, loo, prism, focus, bro, lens, lace, churchman, expansion, pavilion, ointment, tun, park, steel, carpenter, corps, sphere, plum, recreation, gratification, fluency, square, aurora, richness, pearl, ax, primrose, sis, nut, maple, faithfulness, afternoon, rye, ger, mademoiselle, sou, tin, belle, ling, tea, weaver, board, liter, lime, proficiency, arbour, beet, owner, friday, jeweller, fitness, space, banker, fulness, cup, fertility, altitude, hawthorn, ilk, wool, emerald, fishing, fir, punch, boy, edification, ore, field, cot, butler, gel, tad, chocolate, frock, mountain, pic, fervour, rosemary, flower, porch, cam, talent, bedding, solidity, chi, teen, coming, coal, radius, canal, evening, warren, colour, eon, tot, oil, believer, con, sim, reservoir, student, teaching, poplar, ark, quarter, astronomer, universality, bid, penny, fleet, elm, aerial, credit, schooner, exposition, ivory, eld, app, enamel, walker, basket, globe, stocking, inspiration, artist, success, lemon, ride, widower, tan, summer, composition, muslin, knighthood, atmosphere, excellence, filling, workmanship, cherry, autumn, bow, rendezvous, cape, mead, mount, wood, gardener, plat, bom, model, nightingale, tower, compilation, soap, ascension, distinction, glass, synagogue, tee, brass, tic, lyric, diamond, wharf, bologna, deacon, village, gi, author, road, meadow, mic, week, perception ... (372)                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| +Pos+        | tact, organization, recognition, lack, missionary, training, lending, consecration, trait, musician, critique, command, career, incorporation, accord, value, consummation, enthusiast, learner, par, advent, tone, layman, presentment, boon, follower, conference (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | lecturer, encore, singer, breadth, communication, chairman, goo, contribution, chef, forte, gentlewoman, aid, duo, regeneration, era, intuition, solo, herald, harp, list, gilt, cathedral, niece, guild, graduate, elder, engineer, bond, oratorio, role, jubilee, anthem, household, comp, madame, stipend, tonic, organist, mater, painter, vocabulary, gazette, healing (43)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | toilette, manufacturer (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                | chef, entirety, lecturer, trait, graduate, largo, aid, role, thrift, lending, forte, guild, harp, toilette, accord, fete, mater, baton, consummation, herald, duo, award, advent, incorporation, glee, boon, presentment, presentation, gem, carol, stock, toilet, discoverer, breadth, rarity, flora, doc, repute, gilt, par, comp, tuition, homo, jubilee, communication, capital, advertiser, engineer, stud, cheapness, delivery, equipment, housewife, verity, salutation, unit, drawing, lute, bible, periodical, prodigy, refreshment, stipend, information, circle, guidance, exchange, subscriber, zenith, quota, beau, certificate, ballad, leader, grove, gazette, maker, arc, footing, term, price, order (82)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | goo, learner, atlas, regeneration, cleanliness, deli, naturalist, combination, intuition, enthusiast, gentlewoman, follower, store, illumination, layman, chairman, governess, fount, freehold, value, compass, seminary, evangelist, consecration, critique, command, elder, painter, dairy, company, motto, committee, observatory, embroidery, farming, garland, mason, livelihood, repast, maturity, cathedral, journal, manhood, synopsis, niece, abundance, porcelain, harvest, pastor, vocabulary, sustenance, vine, fountain, stature, elasticity, mathematician, flavour, woodland, microscope, fineness, mu, mineral, fragrance, perfume, treasurer, complexion, nourishment, radiance, brightness, verdure, silver, age, volume, fruit (74)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| +Qual        | irony, solemnity, weariness, scrutiny (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | coming, brandy, lip, breadth, ringing, horseback, chill, contact, flash, afternoon, brick, hurricane (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | precision, hand, water, bucket, head, way, appearance, foot, hall, conversation, countenance, glass, vegetation, lustre, heart, distance (16)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | bouquet, railing, belt, sea, loft, curve, sitting, piece (8)                                                                                                                                                                                                                                   | bouquet, tome, blackness, scrutiny, fern, tool, nabob, tweed, breadth, drawing, ringing, sign, vegetation, irony, sort, corpse, halt, conversation, lying, solemnity, circle, beginning, way, relation, lily, horizon, piece, term, corner, place, walking, part (32)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | lip, holly, bucket, sea, gust, foot, curve, precision, moon, casting, coming, chest, hawk, windmill, chill, space, turnip, heaving, side, flash, moment, action, appearance, table, rigging, spot, citadel, fineness, countenance, brandy, dragoon, glitter, draught, summer, noon, rose, stone, track, likeness, parapet, hurricane, afternoon, sparkle, weariness, brick, suspicion, linen, granite, road, kind, hand, horse, perception, resemblance, lustre, glass, face, earth, bay, iron, gate, century, head, whiteness, shot, camp, brass, chimney, morning, water, woman, weather, visage, break, breath, barge, manner, island, door, distance, wall, ground, anchor, river, presence, soldier, battle, heart, affinity, silver, man, brightness, sentiment, nature, metal, harbour, breast, winter, gold, bottom, boat, horseback, surface, bridge, hour, coast (106)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| +Qual+       | impetuosity, coldness (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | storm, atmosphere, melancholy, comrade (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | figure, light, shadow, perspiration, helm, mould, soot, eye, scene, demon, fire (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | muff, moss, rill (3)                                                                                                                                                                                                                                                                           | rill, demon, sheen, pallor, haze, shadow, lizard, clutch, allure, greeting, lure, perspiration, glen, helm, pane, handling, knight, texture (18)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | scalp, figure, leaf, bark, profile, lance, rainbow, atmosphere, lair, roughness, soot, comrade, flower, light, impetuosity, dragon, cleft, mountain, twig, snow, hardness, touch, colour, daylight, emerald, melancholy, expression, scene, coldness, imagery, scarf, night, approach, humour, dryness, sun, window, beech, hair, storm, blanket, almond, colouring, shore, smell, eye, wind, serpent, marble, frost, grass, tree, nose, coat, fire, sword, imagination, rock, mouth, beauty (60)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Quant       | career, ruler, transportation, incorporation, conflict, consummation, sneer, advent, deduction, confiscation, critique, suppression, licensing, disaster, privation, goodwill, threat, outcry, magnitude, appropriation, checking, default, correction, art, bane, churchman, comment (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | lecturer, vita, encore, echo, indigestion, bishopric, herald, singer, sympathy, niece, partnership, clue, intemperance, bass, feud, rote, quid, cycle, tuesday, victor, relict, rite, anthem, forte, sig, stipend, commissioner, dollar, valuation, lodger, county, serf, argent, summary, viscount, refutation, chef, recitative, aid, panic, gentile, plaintiff, exemption, arbitration, role, closing, solo, eon, reliance, earldom, anarchy, pang, rashness, misrepresentation, habit, limitation, council, embryo, disregard (59)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | accommodation, sherry, chemistry, arrangement, mercy, union (6)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | jib, camera (2)                                                                                                                                                                                                                                                                                | lecturer, rental, advent, numbering, deficit, herald, fuss, consummation, rote, ruler, incorporation, feud, clue, role, rite, flora, goodwill, serf, chef, domain, threat, temp, jockey, confiscation, fare, privation, victor, disaster, sig, lodger, forte, jib, prelude, dismay, partnership, sprinkling, licensing, sherry, attendant, recitative, menu, deduction, aid, extension, closing, doc, papa, banquet, suppression, miser, plea, embryo, stipend, arbitration, beginner, sewer, trip, jus, periodical, harpsichord, music, twin, novelty, elocution, hiring, section, naming, deposit, sneer, joke, minority, toast, jam, counting, score, correction, possessor, vole, card, giver, concord, guide, schedule, vita, rebus, reduction, publishing, transaction, quid, stationer, congress, examination, union, system, guidance, attendance, toil, yacht, subscription, sway, tuesday, renown (102)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dollar, distribution, indigestion, echo, tram, indemnity, vid, valuation, para, suicide, seminary, progeny, establishment, multiplication, checking, cycle, sou, accumulation, appropriation, fob, bishopric, default, auto, annihilation, barony, donation, trespass, panic, pit, ruling, churchman, operating, fiction, coke, peerage, partner, phi, mite, mamma, pantheon, camera, dud, magnitude, commissioner, art, ass, refutation, enumeration, bias, bane, maze, limit, pastor, guardian, taxation, result, critique, pang, gal, flu, ritual, litany, freehold, option, monopoly, pig, delusion, niece, qualification, contract, farming, voting, limitation, omission, clemency, repast, derivation, predecessor, reservoir, knighthood, ham, compilation, education, arithmetic, ox, principality, parent, intemperance, misrepresentation, thankfulness, cardigan, comment, outcry, surplus, foresight, inventory, metropolis, summary, virgin, hen, chemistry, relict, committee, gentile, chi, radius, eon, offspring, reliance, viscount, sedition, purchaser, connexion, curse, plaintiff, soup, syntax, council, amp, rem, bill, argent, exemption, division, sinner, number, odium, county, dividend, university, earldom, forgiveness, journal, outrage, imposture, lawyer, pontiff, rashness, imprudence, invitation, majority, disregard, jurisdiction, woe, immortality, supplement, compensation, dictionary, folly, anarchy, redeemer, redemption, acid, freedom, repentance, mercy, borough, vengeance, claim, greatness, chronology, bliss, ignorance, sum, benevolence, guilt, appendix (167)                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| +Quant+      | congratulation, endowment, organization, veto, absolution, ego, training, lunacy, case, mania, boon, axiom, inaccuracy, consecration, archdeacon, ordination, layman, advert, dealing, retention, compulsion, synod, delegate, philology, abbe, indictment, formula, purgatory, convocation, license (30)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | abstinence, derby, regeneration, reservation, opus, list, duo, remission, diocese, era, monday, cheque, priesthood, burglary, dean, inquest, sine, hierarchy, banker, divorce, concession, stoppage, sunday, misconduct, unwillingness, friday, justification, oratorio, chairman, choir, commune, data, willingness, labor, guild, universality, denial, widower, fusion, genesis, barrister, implication, graduate, exclusion, circuit, extinction, poultry, broker, lunatic, rabbi, turkey, organist, budget, promoter, robbery, infallibility, deanery, expansion, tonic, epidemic, stability (61)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | fee, agency, equity, tributary, machinery (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                                | veto, draft, cheque, mania, congratulation, boon, endowment, stoppage, temperance, advert, legation, commune, largo, labor, ledger, directory, scrip, derby, recording, assemblage, adult, data, lunacy, sine, duo, graduate, mining, tributary, holder, rating, award, guild, budget, stake, examiner, retention, abstinence, fellowship, cess, delegate, dating, persona, unit, confederation, enlargement, stud, racing, lunatic, turkey, umpire, digest, asylum, notoriety, codex, sequence, cation, extraction, donor, participation, antiquarian, tuition, exam, opera, chili, ego, formula, promotion, probation, birthday, transit, presentation, disuse, inauguration, tithe, competitor, divorce, couple, teacher, passenger, pedigree, messiah, speculation, manual, incentive, usefulness, servitude, transport, compulsion, school, opus, wage, intro, advertiser, executive, promoter, survey, bookseller, abbe, task, vacation, monday, genus (102)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | fusion, regeneration, reservation, remission, oversight, layman, reviewer, complication, governess, claimant, willingness, concession, suffrage, area, axiom, solving, packing, sis, formation, store, koran, unwillingness, inaccuracy, fart, orphan, sos, production, extinction, inquest, hierarchy, observatory, foreman, sufficiency, librarian, stability, universality, misconduct, banker, implication, tutor, cession, license, ordination, dowry, sunday, proficiency, rumour, client, bribe, exclusion, passport, solicitor, recreation, birthright, interment, consultation, philology, consecration, chairman, unanimity, stat, commemoration, widower, expansion, absolution, attainment, source, dean, spelling, priesthood, epidemic, independence, munificence, fitness, denial, rabbi, teaching, equity, basis, synopsis, livelihood, genesis, indictment, algebra, astronomer, barrister, boiler, bribery, synagogue, broker, diocese, anniversary, cop, prosperity, imposition, ton, atonement, friday, acceptance, fallacy, sustenance, drunkenness, sanction, liberality, petition, advancement, archdeacon, eucharist, rule, conversion, deputation, lexicon, convocation, purgatory, compendium, embassy, adultery, deanery, negligence, progress, catechism, infallibility, commandment, accession, laity, food, validity, justification, propagation, epitome, mathematics, treasurer, conjecture, decree, presumption, founder, sacrifice, synod, redress, fraud, principle, baptism, tribunal, creation, excommunication, inference, famine, definition, essay, grandson (150)                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| +Sing        | tact, exaggeration, annoyance, organization, missionary, rebuke, hostility, temperament, sphere, conflict, axiom, inaccuracy, confiscation, sneer, hearing, verdict, exposition, failure, arrest, allusion, value, mood, agitation, enthusiast, emotion, deception, lack, peril, interpreter, warning, license, massacre, recoil, comment, bankruptcy, heroine, imbecility, art, relapse, accomplice, menace, magnitude, outcry, defiance, attitude, intrigue, malady, conference (48)                                                                                                                                                                                                                                                                                                                                                                                                                                                             | burst, breadth, elder, gallery, chairman, herald, explosion, storm, era, blight, blasphemy, ache, intellect, remonstrance, tinge, niece, workhouse, rise, lamb, street, explanation, goldsmith, thrill, league, tempest, evening, negro, revolt, sine, lace, harp, walker, warren, agony, chill, misrepresentation, panic, cathedral, chef, echo, victor, discouragement, evasion, chapel, embryo, familiarity, creed, flash, abbey, penny, vicar, theory, concession, sink, ride, tea, movement, painter, saint, wreck, biographer, lunatic, mount, conception, nightingale, lease, bond, parish, termination, relaxation, collector, divorce, moralist, allowance, commentary, fox, pang, index (78)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | accommodation, promise, succession, world, dress, instance, hand, light, family, shadow, example, perfection, attention, discovery, way, father, companion, interest, magistrate, search, head, protection, blessing, mirror, name, stranger, creature, nave, station, conversation, argument, heart, doctor, notion, injury, cage, government, capacity, concert, image, manufacture, title, speck, trail, reputation, arrangement, lawn (47)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | sitting, cottage, spring, scroll, cider, skeleton, razor, screen, manufactory, diver, piece (11)                                                                                                                                                                                               | annoyance, desert, herald, uniform, chef, hostility, host, bachelor, transfer, ache, confiscation, glee, sine, harp, substitute, embryo, cider, goldsmith, negro, award, occupation, stool, lunatic, mist, stem, fairy, antiquarian, agitation, fellowship, wanderer, salutation, shadow, lust, wearer, suffering, rarity, manufactory, mood, guise, theme, imbecility, tailor, thrill, moat, victor, breadth, aide, nursery, doctor, exclamation, mystery, memorandum, job, peril, trail, stranger, discoverer, remonstrance, cottage, stab, promotion, vocation, brute, possessor, apparition, tribune, firm, pantomime, domain, diver, contingent, sergeant, companion, bible, workhouse, sensation, cabinet, speculation, magistrate, remnant, deposition, trot, tiger, collector, sneer, trip, display, moralist, sport, refreshment, discouragement, toast, confederation, waiter, vision, protest, reduction, banquet, transition, score, equivalent, novelty, joke, bust, peasant, saving, whisper, novice, waste, charm, transport, presentation, legion, statuary, band, remittance, drawing, sire, attendant, opening, decline, spaniard, fiddle, warning, bull, squire, hearing, web, pedigree, commendation, couple, intrigue, axe, task, seaman, troop, street, advance, supper, counter, chance, strike, relaxation, abode, sufferer, villa, felon, pledge, survey, guide, attention, section, leader, mark, trumpet, attendance, victim, salute, labyrinth, flute, step, mitre, search, generation, pencil, emergency, challenge, divorce, conversation, genus, base, lily, van, date, trifle, inventor, burden, heroine, subscription, idiot, plot, avail, example, traveller, examination, way, circle, engraver, bargain, ballad, home, demand, dame, game, descent, correspondent, society, place, veil, experiment ... (207)                                             | reviewer, arrest, fob, distribution, leech, blight, recoil, skeleton, razor, hook, mound, keel, exaggeration, sting, holly, establishment, lamb, deception, slice, axiom, rebuke, combination, tinge, mirror, verge, oversight, illumination, illusion, shipwreck, naturalist, scoundrel, inaccuracy, encampment, foam, crack, license, mill, glare, ghost, romance, creed, hood, howl, enthusiast, result, panic, suffrage, removal, calendar, chasm, suicide, elder, chest, peep, drama, grin, dash, diary, damsel, turf, rush, impression, buckle, robin, fashion, concession, lace, sphere, passport, monster, plantation, promise, staple, biographer, chill, coal, seminary, chairman, stroke, lot, area, murmur, belle, profession, ass, explosion, rascal, owner, partner, fox, interpreter, boar, piazza, repetition, trap, value, theory, painter, comfort, consultation, pleading, cradle, verdict, store, shock, huntsman, pavilion, pendant, contract, dress, tenure, causeway, flash, driver, menace, conclusion, bolt, distinction, pastor, reserve, unity, devotion, art, sweep, housekeeper, brake, delusion, cotton, bankruptcy, evasion, pilgrimage, misrepresentation, fiction, explanation, cup, instance, sty, injunction, echo, border, massacre, viceroy, witness, fount, madman, warren, basis, amusement, tract, reply, perch, innkeeper, representation, astronomer, accomplishment, calling, goose, taxation, gallop, frock, penny, boy, folio, recreation, beggar, scarf, snow, emerald, burst, comment, lodge, track, growth, dragoon, ladder, reference, employment, sculpture, reflection, walker, talent, boiler, enjoyment, risk, preparation, shape, model, ascent, contest, nightingale, mistake, republican, magnet, field, refinement, overflow, artist, light, landlord, monopoly ... (720)                                                                                                                                                                                                                                           |
| +Sing+       | tone, case (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | park, cloth, cross, amount, list, dinner, sound, castle, fact, square, feeling (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | style, effect, study, soul, idea, appearance, glass, party, heaven, scene, meaning, union, application, distance, pleasure (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                | capital, rest, sort, opera, race, system, press, school, cost, union, season, class, character, saying, labour, memory, report, stage, theatre (19)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | sound, cross, object, square, space, park, action, moment, table, party, extent, effect, style, grave, sight, rose, side, appearance, period, practice, scene, property, collection, expression, act, castle, library, voice, science, glass, idea, summer, village, dinner, stone, change, hair, sun, rate, wood, master, study, palace, soul, meeting, road, subject, health, professor, man, iron, door, reading, college, food, story, distance, evidence, century, description, volume, heaven, respect, influence, question, taste, meaning, girl, mother, pleasure, husband, colonel, beauty, secretary, attempt, language, evil, marriage, book, winter, bishop, force (82)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| +Space       | solemnity, greed (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | exclusion, gateway, envoy, speech, council, hurricane (6)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | accommodation, affair, kingdom, design, majesty, precision, modelling, throne, bed, hall, shaft, fringe, concern, stack, hive, foot, king, fragment, person, litter, resolution, pail, visitor, bit, speck, mould, favour, hoop (28)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | mud, recess, kilt, stall, strip, loon, divan, muff, belt, scroll, sitting, rill, skeleton (13)                                                                                                                                                                                                 | strip, recess, divan, kilt, modelling, greed, masonry, grit, lass, pail, rill, allure, confederation, tweed, ledger, suspension, equipment, litter, affair, media, gateway, section, relation, information, bench, solemnity, ottoman, fragment, minority, step, neutrality, beginning, writ, corner, peace (35)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | shaft, specimen, skeleton, fringe, causeway, fir, stack, bit, chasm, kingdom, horseman, court, hoop, exclusion, side, foot, ass, speck, lair, majesty, deviation, lobby, sultan, cleft, folio, speech, lava, granite, boundary, elector, concern, slope, precision, cardinal, purpose, congregation, scroll, boot, majority, chamber, parapet, chin, nobility, nation, ordnance, scarf, trough, utility, hurricane, city, bed, council, assembly, century, resolution, promontory, rampart, desire, cornice, throne, scheme, thicket, excellency, province, king, declaration, favour, apartment, camp, highness, parliament, ad, consideration, treaty, person, objection, sheriff, coast (78)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Space+      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | highway, narrowness (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | material, spray, ledge, tank, cairn, soot, carpet, tent, rim, ice, squirrel, tarn, paw, dolphin, cage, seam, jet, tributary, pea, jar, slit, kite, rib, urn, rug, cot, junction, nave, bulwark (29)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | peak, railing, slate, patch, jelly, screen, fence, frill, moss, jib, disc, filter, stile, flue, loft, plaid, gear, cone, punt, sea, hip, monkey, zinc (23)                                                                                                                                     | filter, flue, tank, vantage, disc, ledge, frill, raft, patch, jet, cairn, jar, tool, tent, font, tributary, tuft, sewer, reef, zinc, gorge, lizard, haze, inlet, jib, seam, gear, barrier, punt, ice, clutch, pool, fern, span, paving, stair, rib, dais, dolphin, glen, monkey, paw, tarn, stoop, communicating, car, squirrel, stem, cone, shark, rug, incline, nook, fore, gutter, labyrinth, dive, bulwark, row, bank, access, brink (62)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | fen, carpet, hilt, soot, plaid, chip, spray, sill, rim, kite, slit, narrowness, pea, effigy, sea, urn, cot, space, marsh, cage, woodland, tower, fortress, body, shore (25)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| +Strong      | conflict, exposition, culture, endowment, advent, incorporation, fervour, command, training, synod, trait, ego, delegate, critique, boon, follower, ordination, formula, mover (19)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | encore, singer, opus, infusion, pint, street, goal, mall, bass, convulsion, solo, sympathy, intuition, nerve, note, link, chairman, ballot, role, finale, concession, volunteer, intervention, engineer, region, advocate, bid, echo, friday, promoter, biographer, baker, rite, fox, stability, argent, ear, labor, sharpness, flash, coffee, vindication, hierarchy, valuation, recitative, rider, harp, arbitration, occult, painter, consistency, fact, acre, corps (54)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | filly, precision, fee, agency, zeal, jet, sherry, eye, toilette, series, nave, perseverance, knowledge, paper (14)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | jib, filter, razor (3)                                                                                                                                                                                                                                                                         | role, advent, jet, legation, convulsion, filter, labor, culture, baud, fare, deficit, endowment, jib, toilette, trait, theme, forme, quadrille, lye, toast, boon, glee, charm, spark, incorporation, celt, note, equipment, tsar, rite, fan, confederation, temp, ballot, toilet, delegate, fete, presentation, reef, fore, aide, joke, novelty, mover, opera, tune, flora, handling, adult, car, host, dye, gelding, spectator, recitative, row, sharpness, jig, racing, engineer, harp, sherry, pageant, formula, firm, intervention, tug, foe, bookseller, section, gem, executive, repository, doc, volunteer, evacuation, paint, fairy, trump, extension, promoter, opus, arbitration, harlequin, street, beau, leader, flute, manual, incentive, grove, trot, survey, stationer, hail, strike, statuary, information, bust, ego, examination, apparatus, salute, fiddle, labour, character, exhibit, rebus, game, bard, wit (111)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | cleanliness, reviewer, vert, hook, infusion, follower, hobby, stability, file, rowing, fox, pad, steadiness, concession, mill, biographer, razor, booth, intuition, observatory, opponent, combination, advocate, fork, rider, valuation, occult, astronomer, formation, pact, enamel, production, demonstration, recipe, baker, legislator, beet, sweep, whit, fervour, bid, fuel, bow, pen, fir, foreman, ruling, region, horn, drama, aerial, flash, trout, consistency, dash, echo, liberality, chairman, causeway, coal, repast, ram, nerve, hue, tan, painter, predecessor, precision, committee, fable, exposition, ear, tilt, accomplishment, physics, command, preparation, chariot, pin, witness, company, shape, hierarchy, attainment, admiration, instrument, spear, unity, warrior, plumage, munificence, hammer, pack, tin, ordnance, plume, beneficence, touch, jeweller, architecture, defender, flu, critic, lyric, flavour, countryman, pan, corps, critique, blaze, chi, knighthood, ordination, flow, abundance, ton, ore, ladder, perseverance, knot, stat, wave, industry, qualification, view, brush, mason, printer, telescope, mu, success, cane, tincture, paper, dance, torch, song, roar, turtle, flame, breeze, friday, comedy, root, collection, complexion, independence, ivory, schooner, ability, arsenal, zone, pint, whip, tooth, humour, summit, emblem, servant, argent, harness, stature, syntax, acquaintance, poet, understanding, arrow, glow, mix, mom, delicacy, firmament, age, eye, dish, admirer, accuracy, proverb, sex, lyre, harm, acre, integrity, clearness, prowess, publication, synod, wig, tongue, impartiality, work, dexterity, pistol, cream, delight, claret, sky, quart, testimony, crop ... (213)                                                                                                                                                                                                                                                                                                              |
| +Strong+     | tact, organization, canning, ruler, goodwill, career, veto, recognition, dealing, musician, consecration, supporter, fluency, retention, regime, delineation, sphere, missionary, interpreter, champion, advert, philology, axiom, value, sect, art, learner, churchman, element, reformer, courtesy (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | lecturer, regeneration, intellect, era, array, urge, list, builder, graduate, cycle, focus, thinker, aid, derby, guild, partnership, movement, chef, commune, expansion, universality, freeman, interpreting, fusion, comprehension, organist, walker, tonic, data, circuit, jubilee, conception (32)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | merit, style, machinery, player, manufacturer, reputation, elegance, neatness, tributary, study, species (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | rifle, pattern, disc, zinc, camera (5)                                                                                                                                                                                                                                                         | canning, goodwill, draft, veto, graduate, watt, lecturer, commune, baton, temperance, advert, chef, supporter, guild, ruler, urge, disc, rally, retention, jockey, regime, player, thinker, aid, umpire, tributary, numbering, zinc, entirety, discoverer, stud, lever, sect, operator, competitor, mining, array, award, derby, data, rating, assemblage, repute, prefect, directory, partnership, coup, delineation, builder, advertiser, fellowship, challenge, periodical, music, display, banquet, donor, team, antiquarian, media, unit, band, digest, heating, pledge, jubilee, teacher, race, memorandum, commendation, guidance, guide, school, applause, speed, aim, bar, task, gallantry, hull, tuition, china, possessor, maker, performer, inventor, sway, subscriber, experiment, renown, citizen, avail, praise, printing, price (95)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | gauge, regeneration, naturalist, atlas, fusion, pendant, reformer, freeman, cycle, learner, hooker, modification, steel, expansion, pantheon, bias, champion, illumination, interpreter, universality, workmanship, camera, art, churchman, proficiency, magnet, talent, student, focus, excellence, axiom, cable, indication, neatness, electricity, consecration, courtesy, mechanism, trophy, prism, politician, style, fluency, practitioner, mercury, merit, ingenuity, evangelist, compass, motto, philology, microscope, value, sphere, richness, compilation, pavilion, synopsis, interpreting, education, foresight, comprehension, solidity, bayonet, fertility, quickness, weapon, mathematician, reputation, caution, agility, composition, statesman, nicety, gun, globe, walker, discernment, teaching, reservoir, professor, species, taste, pump, inspiration, science, patriotism, conception, sobriety, boldness, triumph, veracity, invention, elegance, sagacity, penetration, journal, probity, wire, cylinder, brass, orator, mineral, lexicon, symmetry, variety, scholar, literature, comet, erudition, historian, frigate, honesty, diversity, pamphlet, shield, skill, metal, monument, study, criticism, patriot, compendium, interpretation, exactness, eloquence, genius, armour, language, size, epitome, writer, definition, tube, treatise (135)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| +Time        | irritation, organization, temperament, irritability, cessation, supporter, allusion, endowment, sneer, value, annoyance, enthusiast, lifetime, tone, detention, relapse, failure, stagnation, follower, art, strife, delegate, emphasis (23)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | burst, explosion, cloth, brandy, echo, anticipation, tinge, feeling, nerve, blight, movement, feud, decease, thrill, longing, valuation, levity, liver, vita, rapture, harp, mutton, trash, reservation, terrace, ride, role, retribution, bid, valentine, lamb, tea, stoppage, pint, atmosphere, chill, apoplexy, pang, embryo, flash, biographer, habit, madame, week, entry, rapidity, transformation, labor (48)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | sherry, filly, performance, arrangement, strength, exertion, concert, constitution (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | wreath, camera (2)                                                                                                                                                                                                                                                                             | supporter, role, stoppage, irritability, watering, endowment, feud, bachelor, lifetime, annoyance, sherry, anticipation, harp, nursery, longing, rehearsal, cessation, labor, rouge, retribution, flora, fuss, gathering, embryo, thrill, ven, dismay, inauguration, digest, detention, birthday, sneer, transformation, chime, trash, opera, mining, trump, zenith, champagne, delegate, harlequin, paint, messiah, fop, disuse, probation, vent, fossil, china, emphasis, beau, executive, tuition, attire, whim, certificate, season, toil, ballad, entry, vita, renown, strife (64)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | blight, valuation, camera, store, tinge, fresco, liver, follower, scent, irritation, enthusiast, echo, explosion, stagnation, valentine, baking, bravo, reservation, revival, brood, interlude, lamb, mourner, drug, biographer, value, cherry, ferment, glare, festivity, specie, plum, temerity, apoplexy, nerve, bid, bile, diary, commemoration, chill, cup, burst, art, whirlwind, pang, modification, fashion, pavilion, acidity, growth, atmosphere, property, renewal, composition, colour, excellence, flash, enamel, bottle, plumage, cookery, tenure, liquor, brandy, affliction, goose, wedding, existence, connexion, relapse, talent, pen, painting, ride, mixture, syrup, blush, alkali, hair, tincture, flavour, cadence, brain, tea, mixing, kiss, zephyr, anniversary, soap, odour, cleaning, brigade, discord, decay, ginger, wreath, disquisition, prospect, twilight, splendour, vibration, pudding, independence, mix, venison, flesh, sparkle, strain, diminution, glitter, contemplation, week, rapidity, perfume, smell, invention, colouring, fermentation, taste, honey, complexion, levity, science, sulphur, pension, fright, shower, offspring, juice, bud, boil, constitution, pressure, gale, delight, blossom, glow, mu, sweetness, nourishment, mutton, strength, bloom, skin, fragrance, gleam, wax, fruit, mind, hunger, brightness, blaze, despair, oats, quart, food, decease, rapture, climate, evaporation, breeze, pound, acid, intercourse, vivacity, beef, frost, powder, stomach, wine, pint, analysis, vinegar, greatness, gold, gallon, dew, inflammation, extravagance (179)                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| +Time+       | regime, career, training, privation, advent, threat, critique, mania, mood, agitation, default, delirium, longevity, canon, weariness, mover, interruption, malady (18)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | encore, childe, lecturer, indigestion, era, herald, bishopric, finale, episode, singer, duo, goldsmith, discouragement, rite, debility, budget, gladness, termination, punch, reckoning, intoxication, aurora, rote, derby, sales, closing, note, partnership, meal, meantime, household, chef, intemperance, allowance, infatuation, victor, unhappiness, cycle, relaxation, stipend, suggestion, organist, premium (43)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | chemistry (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | spring (1)                                                                                                                                                                                                                                                                                     | regime, childe, advent, budget, lecturer, meantime, privation, mania, fete, legation, herald, threat, debility, carol, rite, glee, goldsmith, chef, duo, ascendant, rote, effusion, sensation, novelty, theme, agitation, mood, closing, elocution, mover, racing, discouragement, note, pantomime, plight, bidding, derby, prodigy, victor, omen, quadrille, dating, pageant, criterion, suggestion, banquet, music, occupation, partnership, reckoning, novice, sport, correspondence, teacher, masque, joke, toast, stipend, relaxation, papa, periodical, longevity, troth, generation, dissolution, prodigal, leader, adieu, retirement, guest, haste, luck (72)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | indigestion, seminary, dram, punch, default, episode, drama, bishopric, sib, eve, sales, frolic, lull, drunkard, production, scandal, autumn, gaiety, lyric, demonstration, cycle, aurora, mime, critique, mamma, preparation, impression, chemistry, allowance, unhappiness, gladness, practitioner, delirium, farce, farewell, canon, intoxication, parting, ascension, infatuation, malady, repast, cruise, slumber, observance, muse, bride, waking, weariness, dissipation, option, triumph, dance, digestion, dawn, appetite, intemperance, interruption, adversity, literature, midnight, attack, remembrance, dream, maturity, nurse, bliss, spring, idleness, temper, consumption, mirth, sleep, vengeance, remorse (75)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Vice        | irritation, threat, allusion, disaster, absence, irony, accomplice, delirium, semblance, checking, massacre, warning, visitation, digression, defiance (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | burglary, discouragement, papacy, disapprobation, distrust, narrowness, disregard, refutation, reprobation, slaughter, panic, parody, tempest, epidemic, antipathy, explosion, explanation, chill, divorce, celibacy, apoplexy, scepticism, paradox, impeachment, infirmity, wail, earthquake, termination, penitence, extinction, reconciliation, robbery, speech, stoppage (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | security, inclination, occasion, intention, safety, succession, thing, neighbour, concern, throne, posterity, instance, mercy, monarch, notion, reason, idea, king, glory, preservation, protection, prejudice, performance, happiness, business, soul, argument, event, shadow, magistrate, satisfaction, empire, wife, equity, zeal, bed, countenance, family, example, constitution, meaning, judge, creature, end, answer, rat, government, conversation, scene, darling, fragment, companion, heart, providence, soot, mankind, agency, person, elf, frame, species, resolution, moth, cage, bulwark, doctor, courage, relief, bosom, stranger (70)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | muff, stile, jelly, moss, belt, slate, strip, bite (8)                                                                                                                                                                                                                                         | threat, barrier, wail, discouragement, stab, mystery, strip, disaster, papacy, reprobation, curb, semblance, shadow, puzzle, stoppage, absence, speculation, plea, darling, gutter, blackness, base, lure, magistrate, elf, preservation, irony, rouge, doctor, sensation, protest, wanderer, system, felon, gesture, thinking, companion, safety, bulwark, administration, juncture, adventurer, divorce, stranger, decline, occasion, sort, warning, fragment, conversation, bourbon, frame, riot, transition, memory, intent, minority, defending, antagonist, step, business, state, transaction, crisis, resort, separation, example, diversion, burden, judge, correspondent, curiosity, refuge, making (74)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | howl, progeny, coalition, accumulation, opponent, rack, narrowness, checking, disapprobation, panic, carnage, charge, shipwreck, rat, kicking, lament, removal, rush, epidemic, merriment, parody, ghost, caprice, festivity, maze, refutation, drowning, procedure, vermin, fable, existence, paradox, accomplice, frequency, dirt, chill, bite, concern, apoplexy, irritation, blemish, protection, commonwealth, soot, vigilance, repetition, temptation, extinction, detriment, moth, neighbour, countenance, indulgence, instance, equity, government, thing, contest, circumstance, inability, soliloquy, humanity, exultation, massacre, toad, distrust, inclination, unanimity, answer, antipathy, impudence, pigeon, conclusion, posterity, event, prejudice, visitation, celibacy, whirlwind, delirium, security, deviation, revolution, clemency, consciousness, slaughter, trap, disadvantage, impeachment, employment, rag, detestation, wife, recovery, imitation, scene, pardon, extent, explanation, explosion, rule, timidity, infirmity, prevention, robber, idea, caution, reproof, blush, partiality, artifice, politics, intention, rabble, interposition, adulation, passion, englishman, glory, diminution, fallacy, disregard, crying, relief, monarch, king, ague, despotism, fright, tendency, reparation, adversity, throne, deed, speech, paganism, multiplicity, end, scruple, dungeon, antidote, revenue, remembrance, swine, grave, legislature, people, temper, behaviour, commotion, abhorrence, shortness, villain, plague, indifference, abdication, notion, recourse, decay, tumult, solitude, empire, defect, murder, inequality, species, presence, decency, scepticism, digression, affliction, constitution, satisfaction, astonishment, presumption, creature, adherence, bed, visage, defiance, killing, reason, apology, belief, soul, property, innovation, sacrifice, pestilence, realm, thief, envy, trade, contradiction, disdain, contempt, delay, continuance, tragedy, satire ... (327)                                    |
| +Vice+       | deception, desertion, privation, sacrilege, annoyance, coldness, loathing, inaccuracy, confiscation, penalty, failure, distraction, mania, strife, weariness, imbecility, relapse, mortification, wretchedness, sneer, peril, irritability, carelessness, menace, detention, interruption, recoil, disorder, taint, intrigue, stagnation, contagion, conflict, chagrin, malady, outcry, bankruptcy, greed (38)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | perversion, retaliation, trash, bitterness, inconsistency, blasphemy, rudeness, remonstrance, intoxication, fanaticism, levity, feud, disaffection, banishment, evasion, revolt, misrepresentation, intemperance, mockery, rashness, insensibility, retribution, unhappiness, prohibition, infatuation, anarchy, vehemence, impertinence, blindness, pang, bloodshed, perplexity, meddling, sob, hardship, agony, schism, blight, melancholy, humiliation, baseness, misconduct, violence, atheism, retort, sink (46)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | blast, affair, consequence, reptile, wretch, injury, perspiration, demon, malice, herd, beetle, cur, trail, snail (14)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | mud, railing, fence, mule, rake, patch, monkey, crocodile, spider (9)                                                                                                                                                                                                                          | demon, mule, sob, mania, privation, annoyance, feud, rake, reptile, loathing, desertion, imbecility, spy, trail, humiliation, retribution, incredulity, freak, trash, plight, greed, pallor, confiscation, suffering, yawn, lust, poisoning, sham, retort, peril, patch, remonstrance, clutch, inoculation, monkey, sufferer, exclamation, dismay, disuse, perspiration, vagrant, brute, mist, stain, nuisance, apparition, intrigue, irritability, chagrin, servitude, rout, chatter, consequence, mire, medley, prevalence, miser, opium, detention, brand, herd, affair, crow, dilemma, nonsense, vagabond, gaming, filth, sneer, taint, waste, labyrinth, joke, idiot, exit, snare, strike, victim, whisper, vent, rescue, quack, cell, foe, strife, hazard, dissolution, haste (88)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | deception, retaliation, detection, recoil, fanatic, fatality, puff, annihilation, scoundrel, irregularity, bully, bloodshed, drunkard, grin, dupe, blast, meddling, menace, malignity, gust, glare, carelessness, complication, hush, sting, suicide, hiss, sacrilege, impropriety, temerity, rust, blight, evasion, delusion, puppy, jest, monster, inaccuracy, impertinence, disgust, gasp, stagnation, perversion, madman, pang, scandal, scourge, propensity, rascal, silence, soldiery, prohibition, misrepresentation, distraction, drug, swearing, unhappiness, crocodile, snake, venom, invader, lamentation, mask, indignity, barbarism, blunder, contention, beetle, mortification, destruction, pique, bubble, snail, catastrophe, assassination, insensibility, fanaticism, blindness, rant, beggar, knave, moment, laughter, moan, witchcraft, sloth, trick, murmur, disaffection, rudeness, swarm, intoxication, misconduct, indiscretion, infidel, profligacy, slander, groan, revolt, weariness, brood, mockery, relapse, adversary, jargon, brutality, devastation, infatuation, derision, hypocrite, imposition, obscurity, hardship, uncertainty, rap, complaint, heron, disturbance, spider, malady, disorder, frenzy, blasphemy, penalty, spleen, mistake, omission, alarm, frown, reproach, inconsistency, offender, ferocity, malice, baseness, confinement, grievance, bitterness, wretch, desolation, licentiousness, rage, deceit, owl, chaos, infidelity, bankruptcy, melancholy, wretchedness, coldness, libel, dissipation, fool, iniquity, bribery, affront, curse, negligence, heap, deformity, depravity, imposture, pretext, distress, discord, rashness, mummy, intemperance, expulsion, vexation, invective, interruption, contagion, odium, coward, banishment, doom, imprudence, falsehood, rash, disgrace, disguise, sedition, fate, traitor, meanness, risk, superstition, impunity, conflagration, terror, injury, amazement, popery, scorn, crush, obstinacy, combat, darkness, shame ... (351)                                     |
| +Virtue      | delineation, drawback, culture, fluency, exposition, command, art, sphere, supporter, grace, churchman (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | cloth, tuesday, sunday, monday, lane, chant, wednesday, friday, intellect, pint, thursday, street, cross, league, park, derby, fortnight, infusion, tea, october, cycle, coffee, goldsmith, width, strand, coming, horseback, baker, afternoon, saturday, gentlewoman, haven, sig, faithfulness, mile, meal, labourer, earnestness, amount, mall, warren, opus, brandy, butler, eon, app, vita, parcel, childe, expiration, lamb, gateway, aurora, september, peninsula, evening, link, commandant, dwelling, lace, poultry, finale, sharpness, beach, germ, rote, cheque, january, ire, gallery, depth, ride, mutton, argent, bologna, seer, array, manufacturing, liter, february, acre, willingness, gurney, candle, dollar, believer, castle, universality, priory, county, ward, grocer, square, brigadier, tile, bid, wheat, penny, zoo, edification, morn, stability, channel, conception (104)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | water, material, biscuit, sherry, design, cot, glass, pail, satin, nave, lawn (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |                                                                                                                                                                                                                                                                                                | font, drawback, forme, cheque, culture, germ, tome, goldsmith, span, sig, furnishing, delineation, pail, derby, hock, sheen, dais, watering, supporter, roan, gelding, packet, rote, haven, childe, viol, baud, fellowship, teacher, fern, fife, pew, tweed, seer, sharpness, sherry, warehouse, bar, spa, tuition, car, gurney, gateway, donor, van, dwelling, remittance, grace, homo, media, sprinkling, lever, milliner, hull, terra, tilth, nook, row, usefulness, burg, lane, street, card, gathering, loan, commendation, lad, charm, stationer, twin, array, corpus, ruby, champagne, promotion, guidance, glen, possessor, china, beck, cornet, music, steeple, saturday, peninsula, monday, report, hoard, wednesday, villa, lily, tuesday, thursday, opus, birch, congress, labourer, ballad, bookseller, yacht, swimming, receipt, salute, inn, jersey, service, vita, renown, arc, eminence, order, coach (112)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | mill, hook, rowing, holly, sib, cycle, dollar, sunday, baker, keel, yeoman, bio, fur, coo, sailing, dining, cross, lamb, infusion, art, trout, prism, willingness, steel, loo, rainbow, bro, carpenter, nut, gentlewoman, churchman, lens, sip, ling, auto, peacock, fir, porch, kettle, lace, tun, tin, maple, tot, ointment, liter, faithfulness, stability, plum, richness, sou, ax, pavilion, ark, proficiency, ilk, ger, sis, space, fluency, lime, seller, park, fulness, altitude, cot, tea, afternoon, warren, gel, solidity, friday, mare, aurora, butler, sos, hawthorn, pearl, mechanism, rye, primrose, brim, weaver, ore, jeweller, board, square, tad, effigy, coal, owner, command, universality, mountain, cam, fishing, ordnance, bedding, chocolate, fitness, teen, cup, wand, fertility, poplar, canal, elm, sphere, schooner, eon, biscuit, aerial, lilt, tan, quarter, student, eld, sim, chi, quay, plat, teaching, oil, consultation, reservoir, wool, radius, tilt, putt, tower, ire, bow, tackle, carver, fleet, talent, dial, coming, exposition, tee, rosemary, mercury, believer, cutter, enamel, success, recreation, mead, armament, flu, inspiration, pic, basket, bid, beach, cruise, edification, boy, lawn, wharf, muslin, ton, workmanship, beech, lemon, ride, deacon, app, sod, cherry, causeway, ivory, bra, evening, ridge, filling, coachman, tip, yin, snow, electricity, knighthood, compilation, vase, tier, cape, bom, ferry, gun, tic, preparation, brush, spoon, spire, sugar, frock, oat, gi, penny, nit ... (377)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Virtue+     | tact, recognition, organization, lack, missionary, training, consecration, goodwill, incorporation, critique, musician, lending, accord, value, learner, consummation, par, trait, career, advent, presentment, boon, ordination (23)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | lecturer, encore, singer, goal, breadth, thinker, chairman, contribution, mount, forte, goo, chef, builder, era, duo, aid, list, focus, solo, star, marquess, intuition, regeneration, communication, harp, circuit, herald, terrace, guild, bass, anthem, cathedral, oratorio, walker, role, graduate, engineer, expansion, gilt, jubilee, corps, bond, stipend, comp, mater, tonic, organist, madame, gazette, healing, data, surveyor, painter (53)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | manufacturer, toilette (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | bouquet, manufactory (2)                                                                                                                                                                                                                                                                       | chef, watt, goodwill, lecturer, entirety, thinker, baton, role, largo, graduate, thrift, mater, forte, guild, aid, trait, harp, advent, bouquet, accord, incorporation, lending, duo, toilette, award, herald, consummation, presentation, fete, staff, discoverer, doc, star, presentment, manufactory, equipment, gem, stock, par, stud, repute, boon, breadth, carol, toilet, diploma, builder, jubilee, glee, advertiser, comp, flora, cheapness, unit, delivery, engineer, gilt, data, capital, liege, collation, rarity, communication, periodical, stipend, bible, marquess, salutation, drawing, information, zenith, lute, refreshment, leader, quota, certificate, bust, rood, grove, subscriber, gazette, circle, engraver, footing, price, maker (86)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | goo, learner, atlas, pendant, deli, naturalist, regeneration, cleanliness, intuition, combination, tine, illumination, chairman, store, expansion, freehold, compass, focus, fount, evangelist, value, consecration, hooker, corps, beet, critique, emerald, field, motto, dairy, observatory, astronomer, company, walker, ascension, mason, embroidery, committee, synopsis, manhood, telescope, surveyor, painter, maturity, microscope, garland, colour, repast, cathedral, flower, porcelain, palm, elasticity, mount, globe, vine, harvest, abundance, stature, journal, ordination, fountain, myrtle, mu, octavo, sustenance, chart, fineness, firmament, mathematician, woodland, meridian, size, mineral, flavour, professor, lexicon, fragrance, brightness, number, tabernacle, college, treasurer, perfume, radiance, silver, verdure, nourishment, plate, complexion, quarto, volume, orb, latitude, sum, fruit (96)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Weak        | penalty, sadness, martyrdom, loathing, weariness, drawback, failure, distraction, interruption, cessation, solemnity, exaggeration, delirium, detention, wretchedness, default, greed, peril, chancellor, visitation (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | ache, countess, ending, exchequer, melancholy, remonstrance, decease, sob, chill, discouragement, scaffold, dwelling, prohibition, intoxication, wail, agony, entry, rector, inconsistency, clerk, household, convent, retribution, castle, disapprobation, bloodshed, afternoon, priory, cheque, pang, abbey, violence, exemption, earl, speech, atmosphere (36)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | affair, family, happiness, bed, security, interest, foot, prejudice, consequence, throne, mercy, succession, house, safety, exertion, king, visitor, wretch, treasure, carpet, resolution, crown, end, concert, relief, prince (26)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | jelly, bouquet, railing, slate, sea (5)                                                                                                                                                                                                                                                        | sob, bouquet, greed, wail, loathing, ache, drawback, cheque, rouge, nabob, desert, barrier, dwelling, retribution, mystery, partition, cessation, discouragement, remonstrance, yawn, troop, mister, nuisance, vacancy, detention, cologne, deficiency, posture, peril, electorate, vagrant, affair, solemnity, scaffold, wardrobe, consequence, ending, beginning, correspondence, signing, drawing, treasure, asylum, entry, corner, memory, situation, restitution, intent, monsieur, peace, safety, state, diet, repair, bench, separation, victim, message, finding, content, refuge, descent, taking, making (65)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | martyrdom, rust, plantation, carpet, guy, chill, bloodshed, suicide, drowning, default, dirt, penalty, prohibition, distraction, foot, exaggeration, sadness, indignity, hush, lamentation, residue, regency, melancholy, drunkard, weariness, moan, madman, beggar, soliloquy, delirium, indulgence, visitation, pang, sea, solace, rent, sloth, disapprobation, interruption, vermin, profile, impropriety, consciousness, rupture, chasm, inability, hardness, garret, alarm, house, landing, prejudice, relief, court, swarm, crying, propensity, solitude, groan, intoxication, licentiousness, suburb, residence, dungeon, blood, side, child, popery, wretchedness, possession, custom, remainder, speech, pretence, bed, twilight, afternoon, atmosphere, baggage, day, countess, displeasure, island, clerk, realm, shelter, keeper, disturbance, wretch, march, pain, offender, slumber, pretext, castle, affection, presence, dignity, dissipation, superstition, omission, fright, revenue, chancellor, shame, security, family, contrary, odium, behaviour, weeping, insolence, rebellion, enjoyment, disobedience, delay, resolution, importation, grandfather, province, communion, languor, illness, grievance, abhorrence, abbey, burial, throne, end, land, madness, thrust, ferocity, rector, sickness, king, privilege, insult, affront, turning, confession, payment, oblivion, defeat, inconsistency, time, vexation, indifference, refusal, happiness, inconvenience, petition, immorality, widow, breach, horror, money, rage, anger, prince, despair, departure, sinner, bondage, prayer, priory, gloom, agony, empress, decency, imprisonment, tenderness, jurisdiction, decease, noon, contempt, crime, necessity, danger, exemption, convent, torment, sleep, consent, woe, arrival, eternity, princess, scarcity, quarrel, equator, mercy, apprehension, offence, grass, hospital, prisoner, mother, violence, apartment ... (235)                                                                                                              |
| +Weak+       | annoyance, desertion, privation, coldness, irritation, irritability, carelessness, mortification, imbecility (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | longing, coming, bitterness, trash, workhouse, banishment, mourning, unhappiness, hardship, childe, mockery, gaol, perplexity, blasphemy, humiliation, gladness, misconduct, insensibility, apoplexy, levity, habit, penitence (22)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | shadow, wife, perspiration, darling, doctor, malice, demon, heart, father (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | moss, sitting, room (3)                                                                                                                                                                                                                                                                        | annoyance, privation, demon, desertion, longing, humiliation, reverie, wanderer, pallor, childe, shadow, darling, suffering, clutch, irritability, trash, imbecility, doctor, lizard, trance, perspiration, blackness, sufferer, workhouse, incredulity, apparition, pool, bachelor, churchyard, mermaid, room, dismay, chatter, gutter, stain, nursery, mourning, corpse, thinking, filth, felon, vagabond, burden, lying, idiot, brink, home (47)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | carelessness, fatality, shipwreck, unhappiness, apoplexy, dupe, sting, ghost, hardship, verge, mortification, irritation, misconduct, gasp, disgust, finery, wife, heap, insensibility, mockery, complaint, coldness, gaol, silence, coming, dormer, uncertainty, reproach, obscurity, scandal, desolation, gladness, ague, distress, depravity, consternation, confinement, prison, bitterness, banishment, malice, perplexity, infancy, derision, darkness, deformity, slander, loss, disgrace, summer, misfortune, rubbish, idleness, terror, night, blasphemy, brutality, disappointment, fatigue, cabin, negligence, drought, misery, suspicion, straw, infection, neglect, pestilence, amazement, penitent, imprudence, consolation, floor, dread, timidity, infamy, abyss, calamity, poverty, morning, meanness, door, remembrance, parlour, window, funeral, grandmother, impatience, plague, resentment, ingratitude, heart, indolence, rheumatism, hanging, confusion, penitence, levity, father, weakness, uncle, aunt, hour, wilderness, uneasiness, exile, cowardice, affectation, winter, calumny, compassion, wickedness, avarice, extravagance, inflammation, coffin, pillow, death, cruelty, wrath, cousin, sister, repentance, ignorance, joy, guilt (126)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| +Woman       | drawback, import, par, suppression, lending, convocation, exposition, heroine, trial (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | childe, explanation, perversion, faithfulness, tuesday, regeneration, madame, vindication, mademoiselle, divorce, prohibition, creed, exclusion, council, liter, wednesday, index, preface, gladness, expansion, aurora, impeachment, justification, account, genesis, demolition (26)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | precision, hall, performance, material, article, promise, accommodation, intention, design, proof, kingdom, agency, foundation, commerce, arrangement, preservation, prejudice, interest, ion, notion, argument, crown, thing, discovery, reason, answer, chemistry, idea, favour, inspection, antiquity, manufacture, power (33)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | doll, sitting (2)                                                                                                                                                                                                                                                                              | drawback, childe, bible, cologne, watering, doll, import, par, lending, suppression, presentation, proximity, spa, terra, partition, annex, demolition, preservation, opening, electorate, cheapness, divorce, usefulness, mosaic, nursing, ottoman, restitution, statuary, information, tonnage, publishing, section, press, lily, examination, vision, equivalent, chancel, veil, monsieur, neutrality, heroine, peace, diet, demand, state, giving, exhibit, order, price, advertisement, place, tuesday, part, wednesday, academy, arc (57)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | coo, article, solving, creed, regeneration, supremacy, mademoiselle, promise, ting, regency, expediency, liter, constituent, identity, ion, frequency, pearl, prohibition, kingdom, explanation, faithfulness, expansion, staple, renewal, multiplication, exclusion, production, multiplicity, conformity, con, precision, preference, abridgment, fiction, finery, abbess, modification, removal, perversion, lid, rosemary, impropriety, nun, restoration, existence, utility, answer, revival, conclusion, court, cert, thankfulness, morality, communion, matter, impeachment, sanction, prejudice, sequel, aurora, limit, reference, efficacy, muslin, progress, embroidery, preamble, council, holiness, omission, continuation, convocation, foundation, confirmation, introduction, propriety, journal, discovery, rupture, intention, appeal, plan, arithmetic, observance, physics, purpose, gladness, divinity, future, project, manufacture, solution, act, geography, dryness, commerce, thing, sculpture, dedication, chemistry, authenticity, importance, treaty, decision, certainty, genesis, scheme, author, rem, validity, frontispiece, propagation, octavo, nit, conjecture, supplement, exodus, jurisdiction, province, work, rectitude, subject, exposition, proof, variable, reformation, index, favour, collection, trial, catechism, paganism, liturgy, substance, worship, objection, confession, coral, eucharist, declaration, idea, cornice, reading, notion, delay, empress, faith, popery, alliance, evidence, evaporation, consideration, violation, publication, church, paragraph, clause, origin, contrary, interpretation, proposition, petition, importation, june, alteration, antiquity, elasticity, grant, mythology, scripture, tranquillity, diversity, bud, brightness, statute, volume, tie, redemption, necessity, saviour, ordinance, version, diction, quarto, sin, recourse, extract, religion, sentiment, compensation, blossom, ad, preface, dependence, prophecy, power, testimony, proposal, reason, opinion ... (233) |
| +Woman+      | grace (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | household, duo, terrace, convent, unhappiness, sob (6)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | happiness, satin, majesty, goodness, carpet, concert, hive, stocking, elegance (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | bouquet, tress, room (3)                                                                                                                                                                                                                                                                       | bouquet, tress, font, nursery, mystery, rouge, sob, sheen, pallor, allure, knitting, duo, grace, pet, drawing, room, separation (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | carpet, mite, belle, flower, porch, balcony, growth, washing, ritual, refinement, majesty, unhappiness, vase, stocking, mum, hem, cookery, attachment, devotion, poppy, purity, weaving, linen, syrup, happiness, prayer, baptism, bunch, goodness, elegance, convent, gentleness, consent, adoration, altar (35)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| -Ambig       | exaggeration, congratulation, musician, learner (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | judaism, unwillingness, anthem, antipathy, willingness (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | spider (1)                                                                                                                                                                                                                                                                                     | congratulation, informant, knitting, dismay, asylum, pantomime, eminence (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | exaggeration, learner, unwillingness, willingness, bully, judaism, prism, monster, antipathy, spider, gasp, muslin, beech, jacket, inability, pudding, cheek, syntax, march, deck, woodland, gown, thicket, handkerchief, glow, cream, anatomy, magnificence, gloom, constancy (30)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| -Ambig+      | archdeacon, archbishop, chagrin (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | mutton, singer, lace, indigestion, herald, moralist, liver, tempest, widower, vocabulary, parentage, lunatic (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | filly, helm, lustre, sherry, nave (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | screen, peak, recess, rifle, vest (5)                                                                                                                                                                                                                                                          | herald, prelude, recess, helm, moralist, foal, vest, repute, lunatic, craft, harpsichord, chagrin, sherry, parentage, quadrille, ivy, champagne, papa, pedigree, relation, militia, supper, chaise, sway, coach (25)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | leaf, contention, lace, liver, indigestion, trout, hush, rosemary, gin, gust, dialect, daylight, kettle, noun, madman, matron, appellation, mamma, suicide, mast, pearl, table, cup, widower, parrot, shade, vocabulary, diamond, lustre, heroism, chalk, manhood, disquisition, mate, anniversary, knot, archdeacon, voting, pine, coward, mutton, lemon, duchess, velvet, infancy, archbishop, midnight, cough, cedar, adultery, tempest, waistcoat, amazement, acid, grandmother, shoulder, testimony, anchor, laurel, sister (60)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| -Changed     | deception, appropriation, layman, exaggeration, martyrdom, stagnation, hearer, arrest, checking, distraction, rebuke, enthusiast, inaccuracy, reformer, axiom, irritation, follower, churchman, sadness, carelessness, accomplice, sphere, learner, default, fervour, bane, recoil, proclamation, prosecutor, sacrilege, delirium, penalty, verdict, courtesy, consecration, coldness, value, menace, art, critique, hindrance, mortification, relapse, interpreter, contentment, magnitude, abatement, formality, license, impetuosity, indictment, archdeacon, absolution, phraseology, fluency, visitation, impediment, bankruptcy, massacre, ordination, champion, lord, comment, weariness, command, disciple, stress, disorder, canon, reluctance, confessor, exposition, malady, interruption, emotion, outcry, digression, philology, defiance, trial, purgatory, synod, contagion, chancellor, wretchedness, convocation, archbishop (87) | regeneration, retaliation, goo, blight, valentine, lip, indigestion, reservation, liar, liver, democracy, lamb, tinge, cross, infusion, adjournment, mediocrity, bishopric, tariff, fusion, sunday, freeman, rider, unwillingness, cycle, refutation, shoemaker, acquittal, gentlewoman, implication, thesis, prediction, valuation, elder, perversion, intuition, sound, liter, park, echo, exaltation, tenant, epilogue, lace, warren, banker, prologue, ado, deafness, celibacy, judaism, bearer, comrade, dit, exclusion, lint, baker, extinction, panic, evasion, narrowness, chairman, episode, fox, dollar, commissioner, remission, meddling, gall, allowance, bombardment, inquest, explosion, bloodshed, disaffection, occult, gaol, vestry, paradox, chaplain, rudeness, butler, punch, acquiescence, ebb, stability, prohibition, interpreting, distrust, biographer, coquette, inferiority, willingness, inexperience, burst, concession, recorder, disapprobation, condemnation, creed, denial, coroner, ire, flash, diocese, argyle, explanation, faithfulness, believer, universality, expansion, coming, revolt, focus, courtship, mademoiselle, advocate, impertinence, canto, chill, melancholy, comprehension, hardship, consistency, unhappiness, cathedral, assent, scepticism, apoplexy, gladness, misrepresentation, limitation, bid, fanaticism, kinsman, barrister, friday, region, niece, parody, dean, intoxication, nerve, tide, brandy, bologna, pang, sales, theory, blindness, viscount, rector, app, square, aurora, surveyor, antipathy, priesthood, vocabulary, blasphemy, painter, tea, relict, impeachment, widower, broker, bitterness, misconduct, vicar, insensibility, afternoon, epidemic, hierarchy, nightingale, inconsistency, walker, edification, gentile, incarnation, frankness, rabbi, brevity, ear, storm, eon, vault, intemperance, banishment, corps, executor, calmness, penny, week, countess, plaintiff, apothecary, infirmary, depth, reliance, deputy ... (287) | carpet, kite, herring, cob, slit, pea, flight, article, promise, blast, lath, rim, bucket, ion, oar, cove, mat, soot, rat, mirror, snail, onion, dress, figure, kingdom, shaft, spray, slide, gad, bit, ornament, wing, diadem, biscuit, stack, precision, majesty, cot, beetle, moth, neighbour, posterity, protection, appearance, merit, neatness, instance, discovery, concern, countenance, stocking, glove, party, inclination, thing, satisfaction, fringe, light, carving, hoop, security, reel, perseverance, intention, cushion, chemistry, dart, lawn, freight, glory, boa, style, wife, effect, vat, urn, foot, relief, rabbit, event, world, commerce, bowl, empire, toe, broom, comb, pleasure, speck, government, answer, equity, bed, paper, constitution, end, reputation, copy, prejudice, stalk, fluid, assistance, foundation, goodness, support, injury, honour, belly, king, advantage, notion, species, wisdom, happiness, house, idea, lustre, family, capacity, animal, glass, throne, cage, wretch, heaven, hand, study, monarch, scene, malice, title, proof, friend, father, manufacture, grease, head, reason, soul, person, favour, antiquity, mercy, distance, resolution, blessing, creature, perfection, strength, water, motion, authority, power, twine, prince, zeal, elegance, salvation, esteem, name, deity, fire, courage, argument, providence, heart, mankind, bosom, eye, meaning, knowledge, image (172) | plaid, puss, rind, nymph, mug, curve, skeleton, razor, lump, bite, equipage, camera, crocodile, sea, spider, load, morsel, wreath, scroll, spring (20)                                                                                                                                         |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| -Freq        | inaccuracy, lunacy, incorporation, mover, proclamation, abatement, par, digression, abbe, chagrin, delegate (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | childe, acquittal, inexperience, burglary, infatuation, adjournment, brigadier, ado, aid, commandant, evasion, recorder, gentlewoman, ire, rote, retaliation, implication, prohibition, council, promoter, account, executor, apothecary, comp, zoo, bid, vindication, broker, apoplexy, flood, fusion, demolition (32)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | fee, kingdom, fad, king, hive, ion, concern, mutt, affair, design, security, prejudice, prince, cot, cur, mote, majesty, cairn, equity, mat, elf, snail, foot, tarn, gad, twine, bucket, wick, vat, oar, jet, answer, bulwark, hoop, toe, resolution, belly, motion (38)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | muff, railing, belt, kilt, stile, haft, stall, rake, chine, tress, equipage, punt, coon, loon, loft, frill, rill, jib, cone, fort, jade, rift (22)                                                                                                                                             | childe, whore, asset, heft, raft, tome, rill, rime, contra, kilt, legation, allure, font, tress, mover, rake, punt, ration, dispatch, thud, grit, nabob, chine, frill, forme, lye, gene, lunacy, rating, rift, evacuation, rote, monitor, aid, jet, fore, jade, fife, twill, fop, film, prance, inlet, lure, cation, cairn, playhouse, itch, scan, elf, hiring, incorporation, jib, publican, comp, par, persona, wick, trump, posture, fan, cone, mote, tarn, liege, electorate, ex, chagrin, plea, verity, vagrant, demolition, housewife, lotion, delegate, disuse, milliner, survivor, gent, larder, promoter, marl, gull, bulwark, reward, physic, affair, diversion, elk, pant, hatch, restitution, felon, dive, neutrality, halt, quota, repair, foundling, abbe, hazard, intent, peace, arc, order, wit (106)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | ting, fen, tick, chip, char, fice, prom, cult, liner, coo, caliph, bout, trill, gradation, froth, whit, ion, fatality, ranger, oar, mallet, muck, carnage, inaccuracy, pate, fusion, progeny, rant, celerity, temerity, ref, fart, retaliation, pact, guile, pap, ger, bus, phi, pore, ado, equipage, kingdom, tike, scar, fir, maze, putt, gnat, shopkeeper, acquittal, birthright, auto, fem, con, rho, pique, para, subaltern, dice, cleft, pal, puppy, cot, gel, pad, vid, pear, cel, pout, proclamation, oke, inexperience, tinsel, ping, brewing, implication, alloy, steadiness, gland, gad, gunner, mat, deli, multiplicity, gal, turnip, petitioner, adjournment, bucket, kin, evasion, ax, teen, talc, sip, cad, concern, frolic, mercury, snail, solving, windmill, preamble, regency, gentlewoman, lam, roper, prohibition, courtier, rig, armament, bro, infatuation, sot, hoop, smuggling, detriment, equity, interlude, beneficence, chi, abatement, dryness, apoplexy, brigadier, lard, dauphin, brine, dud, ire, bid, fleece, indemnity, foot, pom, rudder, confederacy, ace, knave, batt, amity, unanimity, jeweller, gi, fib, ox, ore, digression, bib, king, prejudice, bail, barge, contraband, affront, mediation, caw, fineness, prince, augmentation, rem, pe, kindling, council, exuberance, bot, cam, allegory, court, crab, cardigan, burnet, broker, tab, rectitude, garret, jargon, ferment, cop, pretence, majesty, mint, fleet, bra, spleen, cure, detestation, toe, answer, infinity, ant, husbandry, adulation, zephyr, invective, tic, var, idiom, retinue ... (284)                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |